### 설명
- t+1 : __신용회복 계좌별진행상황 업로드 후(그래야 신복키나옴) 업로드결과, 입금완료 후 개인회생, 입금내역 다운__
  - 입금 : 등록일 기준 t+1 ~ 입금마감일로 해서 다운. t:기준일 = 계좌새창다운일(다운로드 후 입금등록 없어야 함!)
    입금일이 언제이고 입금내역이 원금이건 이자이건 계좌 금액과 이자 계산 제대로 되었는지 재확인 필요
- 자산정리 월별 차이점 정리
  - 4월 : 확실한 종료(종결, 매각, 환매, 차주면책+정상보증인없음)만 종료로 처리
  - 5월 : 할당제외한 모든 종료,불가사유 종료로 처리(제외, 포기, 시효, [차주 : 사망,이민,말소,파산,법인]+정상보증인유무에 따라 보증인있음 or 종료처리요망)
  - 5월 재악업 : 솔림 비정상건 반영, 솔림 신복 opb에러 수정(대성은 제출 후라 수정 못함)
  - 6월 : 대성 비정상건 반영, 대성 신복opb 수정
  - 8월 : 법조치 새채무자키 전체 적용, 지급명령 인용건, 타채 인용건(해지된 경우 제외) 반영
  - ~~10월 : 기준일 입금 내역 반영(t+1일에 수정(등록포함)한건), 개인회생 실제 총분납잔금 반영(실제 총분납잔금 = 전산 총분납잔금 - 전산 초과입금 = 총분납금 - 총분납입금)~~ 240613 삭제
    - 입금새창에 있는 현재금액에서 t일 회수액을 차감해서, t일에 등록하고 t+1일에 수정한 건도 정확히 반영
  - 24-02
    - (기존) 차주와 보증인이 모두 개회중이거나 모두 신복중인 경우만 보증인 시트 작성 > (변경) 보증인이 조정중인 경우 차주상태와 무관하게 보증인 시트 작성
    - (기존) 일반회생 전산에서 분납중지로 놓았기때문에 무담보로 분류되었음 > (변경) 개인회생으로 분류될 수 있도록 조치
    - (기존) 신용회복 말일 완제자, 다음날 전산 반영하므로 상태 및 담당자 미변경 & 기준데이터에선 완제는 제외 따라서 무담보로 분류되는 문제 > (변경) 계좌,보증인 조회새창 채무상태,담당자 수정으로 해결.
    - (기존) 개인회생 면책건 전산 상태값에 의존 > (변경) 계좌,보증인조회새창 채무상태,담당자 수정으로 해결 with 면책건 파일
  - 24-05
    - (기존) 신복 정식에서 후일입금내역 차감 > (변경) 금액도 일일업데이트되는 약식 + 정식 : 진행상태와 상환후 잔액은 입금내역 반영하지 않아도 기준일 최신화 됨
    - (기존) 후일입금 다건 : 회수합계 가장 큰 거 하나만 남겨, 입금조회파일의 현재금액 또는 회수금액을 활용 > (변경) 다건 삭제없이 모든 후일입금에 대해 계좌의 현재금액을 직접 수정(연체이자도 수정)
  - 24-06
    - (기존) 개회 총분납잔금 = 전산총분난잡금 - 초과입금 > (변경) 전산총분납잔금 그대로
    - (기존) t+2일 자료에 t일 입금내역 반영 > (변경) 개회 입금 자동화로 t+1에 t일 입금내역 반영
    - (기존) 계좌 t일에 다운 > (변경) t+1일에 다운 : t일까지의 입금 모두 별도작업없이 반영, t일 입금으로 인한 상태 및 담당자 변경 별도작업없이 반영, 다만 연체이자를 하루 전 값으로 환원작업 추가. 그달의 마지막 평일이 아니라 말일을 기준으로 컷오프 가능
  - 24-08-23
    - (기존) 실효일 없는 실효(실효예정자) : 실효로 처리 > (변경) 정상이행으로 처리 (전산도 이렇게 자동 업데이트 하고 있음)
      - cf)합의서체결일 없는 합의서체결완료는 상환유예재조정 중, 완제일 없는 완제일은 채무자 입금하였으나 신복 전산처리 미완료상태로 완제 맞음(17시쯤 날짜 찍힘)
    
  
- 추가 할 일
  - 채무자별 법조치 정리파일 만들면 좋겠다. (금감원 보고시에 지급명령등 접수일 필요함)
  - 파산 : 법조치등록건 반영
  - 신용회복실효반영.ipynb : 실효건은 실효통지일(실효일)로부터 시효계산 되는 거 반영

### 작업전 할 일
0. 기준일 담날 모든 데이터 다운 받으면, 신복 특별면책류만 확인하면 됨(개회는 기준데이터 하면서 직접 다운받은 파일 수정)
1. 계좌-면책완납 담당자 종결로 변경, 보증인-면책완납자 보증상태변경 : t+1일 담당자변경로그,보증상태 변경로그통해 계좌 수정, 신복 그주의 특별면책,동시면책, 개회는 기준데이터_면책건으로 크로스체크
2. 계좌 금액 : 기준일 신복 실효건 : 최초원금 ~ 현재합계, 계좌조정일~조정사유, (실효폐지는 기준데이터에서 빠지므로 어차피 무담보로 분류므로 상태값은 수정 안 해도 됨)
   - 신복 입금과 전산입금 동기화 되면 굳이 안 해도 되는 작업
3. (3개월마다) 시효_신용회복실효반영.ipynb 돌리고 와라 (최근작업일 4/30)

### import

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
import warnings
warnings.simplefilter("ignore")
import functions

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format

# dtype 정의
pattern = re.compile('키|코드|계좌|차수')

debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str, '관리자비고':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str, '신고계좌':str, '입금계좌':str}
credit_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
# credit_dtype = {'채무자키':str, '계좌키':str, '보증인키':str, '심의차수':str, '변제금\n수취계좌':str}
# credit_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
# }
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}
약정_dtype = {'채무자키':str, '계좌키':str, '분납키':str}

# 차감할연체이자 계산용
def 날짜차이(입금일:str, 기준일:str) :
    date_format = "%Y-%m-%d"
    """ 문자열 타입의 두 날짜 사이의 날짜차이 리턴 """
    date1 = datetime.strptime(입금일, date_format)
    date2 = datetime.strptime(기준일, date_format)
    return (date2 - date1).days # .days : 정수값 얻기위해

In [2]:
##################################
company = "솔림"      # 솔림 or 대성
basedate = "241031" 
##################################
기준일 = '20' + basedate[:2] +'-'+basedate[2:4]+'-'+basedate[4:6]
# 1년 365일 / 366일?
totalday = (pd.to_datetime(f'{"20" + basedate[0:2]}-12-31') - pd.to_datetime(f'{"20" + basedate[0:2]}-01-01')).days + 1

wd = join(r"D:\3.자산\전산 dataset", company)
comp_closing = {
    # 채권 자체의 소멸
    "종료" : r"(?<!청산)종결(?!요청)|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료",
    # 비정상-종결요청
    "종결요청" : r"종결요청|매각\(예정\)|(?<!피)환매\(예정\)|패소",
    # 시효
    "시효" : r"시효|완성|만료", # 부활 제외함 231222
    # 채무자별로 체크
    "상각" : r"상각", # 면책포함여부보다 먼저 검색되어야 함. 실사한 것인고, 담당자에만 있으므로 차주 파산(확정) 이런 거보다 우선함. 대성에서 원래 상각이었던 거는 내담당으로 바꿈. 솔림대성 모두 실사한 것만 담당자 상각임
    "면책" : r"면책|면탈",  
    "불가" : r"추심제외|포기|파산(?!(\(기각\)|\s누락))|사망(?!\(상속승인\))|폐업|청산종결|패소", # 제외, 포기는 이미 보증인까지 고려한 결과일 수 있음 # 담당자 누락제외는 채무상태에 적용하면 안 됨.(계좌의 메모를 봐야함)
    "공채권" : r"공채권"
    # 이민,국적상실, (말소), 실종, 거주불명, 채권존부소송중은 정상으로
    }

원데이터파일명 = {"개회차주" : "2."+ company +"_개인회생_원데이터_차주"+basedate, 
                "개회보증인" : "2."+ company +"_개인회생_원데이터_보증인"+basedate, 
                "신복차주" : "2."+ company +"_신용회복_원데이터_차주"+basedate, 
                "신복보증인" : "2."+ company +"_신용회복_원데이터_보증인"+basedate, 
                "전체" : "2."+ company +"_전체_원데이터_"+basedate}

In [3]:
###################                                                                             # 개회의 겨우 계좌키 분리 파일로 읽을 것! 
ext = ".pkl"  
###################
파일검색어 = ["채무자조회새창_", "계좌조회새창_", "보증인새창_", r"개인회생새창_.*기준데이터", "신용회복_기준데이터", "법조치조회새창"] # 후일입금 파일 있으면 추가 입력
if company=="솔림":
        파일검색어[3] = r"개인회생새창_.*계좌키분리_기준데이터"
        
# r"보증인새창_.*새채무자키" , "보증인새창_" 중에서 택일

# 기본 경로
base_dir = join(wd, basedate)
if ext == ".pkl" : 
        base_dir = join(base_dir, "pkl")
# 파일검색어
# 파일명리스트
files = []
for i in range(len(파일검색어)) : 
        if re.search("보증인새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],'새보증인상태',전체경로=False))[0])        
        else : 
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],전체경로=False))[0])

In [4]:
# 파일 읽기
# 채무자, 계좌, 보증인, 개회신복 기준데이터->차주와 보증인으로 나누 저장
if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    debt = pd.read_pickle(join(wd, basedate, "pkl", files[0]+ext))
    account = pd.read_pickle(join(wd, basedate, "pkl", files[1]+ext))
    grt = pd.read_pickle(join(wd, basedate, "pkl", files[2]+ext))
    rehabilitation = pd.read_pickle(join(wd, basedate, "pkl", files[3]+ext))
    credit = pd.read_pickle(join(wd, basedate, "pkl", files[4]+ext))
    credit_excluded = pd.read_excel(join(wd, basedate, files[4]+".xlsx"), sheet_name="일부계좌제외", dtype=credit_dtype).fillna("") # 읽는데 시간 얼마안걸리니 별도파일만들지말고 엑셀읽자
    event = pd.read_pickle(join(wd, basedate, "pkl", files[5]+ext))
    if len(files) >= 7 : 
        deposit = pd.read_pickle(join(wd, basedate, "pkl", files[6]+ext))
else : 
    debt = pd.read_excel(join(wd, basedate, files[0]+ext), dtype=debt_dtype).fillna("")
    
    # 새채무자키 없으면 일부러 오류나게
    if len(debt.query("관리자비고==''")) > 1 :
        print("새채무자키 부여 필요")
        print(len(nothing)) # 오류를 위한 코드

    account = pd.read_excel(join(wd, basedate, files[1]+ext), dtype=account_dtype).fillna("")
    grt = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=grt_dtype).fillna("")
    rehabilitation = pd.read_excel(join(wd, basedate, files[3]+ext), sheet_name="개인회생(진행)", dtype=rehabilitation_dtype).fillna("")
    credit = pd.read_excel(join(wd, basedate, files[4]+ext), sheet_name="확정,미확정", dtype=credit_dtype).fillna("")
    credit_excluded = pd.read_excel(join(wd, basedate, files[4]+ext), sheet_name="일부계좌제외", dtype=credit_dtype).fillna("")
    event = pd.read_excel(join(wd, basedate, files[5]+ext), dtype=event_dtype).fillna("")
    if len(files) >= 7 : 
        deposit = pd.read_excel(join(wd, basedate, files[6]+ext), dtype=deposit_dtype).fillna("")


    # 엑셀 읽었을 때 해야 할 작업. 10월부터 아래 두줄은 할 필요 없음------------
    # 엑셀저장시 NaT 를 ""로
    # credit.replace({pd.NaT: ""}, inplace=True)
    # # 신용회복 상환후잔액 타입 int로 
    # credit["상환후잔액"] = credit["상환후잔액"].replace("",0).astype(np.int64)

    # 특수건(예수금, 허은석)제외
    debt.drop(debt[(debt.성명=="예수금") | (debt.채무자키=="20565788")].index, inplace=True)
    debt.reset_index(drop=True, inplace=True)
    account.drop(account[(account.채무자명=="예수금")| (account.채무자키=="20565788")].index, inplace=True)
    account.reset_index(drop=True, inplace=True)
    
    # 법조치 빈칸 제외
    index =  event[(event.관할법원=="0") | (event.관할법원=="") | (event.사건번호=="0") | (event.사건번호=="")].index
    event.drop(index, inplace=True)
    event.reset_index(drop=True, inplace=True)
    # 사건번호 역순정렬
    event.sort_values('사건번호', ascending=False, inplace=True)

    # 채무자 주소 보정
    ### 주소채우기 : 초본이 없는 경우 자택 > 직장 > 기타 주소를 초본칸에 넣기
    # 주소에 공백 제거
    debt["초본주소"] = debt["초본주소"].str.strip() ########## 초본주소가 현 자택주소인 경우가 많고, 자택주소보다 더 잘 되어있음
    debt["자택주소"] = debt["자택주소"].str.strip()
    debt["직장주소"] = debt["직장주소"].str.strip()
    debt["기타주소"] = debt["기타주소"].str.strip()

    # 1) 초본이 빈칸인 경우 다른 주소를 초본 주소에 채우기
    for i, v in tqdm(debt.iterrows(), total = len(debt)) : 
        if v.초본주소 == "" :
            # 초본주소 없을 때
            if v.자택주소 != "" : 
                debt.loc[i,'초본주소'] = v.자택주소 #+ " " + v.자택번지인
                debt.loc[i,'초본우편번호'] = v.자택우편번호
            # 직장주소라도 있을 때
            elif v.직장주소 != "" : 
                debt.loc[i,'초본주소'] = v.직장주소 #+ " " + v.직장번지인
                debt.loc[i,'초본우편번호'] = v.직장우편번호
            # 직장주소도 없을 때는 기타주소로
            else : 
                debt.loc[i,'초본주소'] = v.기타주소 #+ " " + v.기타번지인
                debt.loc[i,'초본우편번호'] = v.기타우편번호

# 조정채권 분리 (충돌 확인 위해 필요함)
# copy() 하면 pd.NaT도 또 해주지 않아도 된다.
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"].copy()
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"].copy()
credit_d = credit[credit["채무구분"]=="주채무"].copy()
credit_grt = credit[credit["채무구분"]=="보증채무"].copy()

# 신용회복 반영OPB
# if 진행구분==확정 : 반영OPB = 상환후잔액 if 상환후잔액=="" : 반영OPB = 조정후합계
# else 반영OPB = 조정전원금

In [ ]:
# # 개별 파일 읽기
# # deposit = pd.read_excel(join(wd, basedate, "입금_계좌다운후.xlsx"), dtype=deposit_dtype).fillna("")
# deposit = pd.read_pickle(join(wd, basedate, "pkl", "입금_계좌다운후"+".pkl"))

In [ ]:
# # 개별 파일 pkl 만들기
# filename = "240108_신용회복_기준데이터_솔림(1)"
# ext = ".xlsx"                                            ############################################
# df = pd.read_excel(join(wd, basedate, filename+ext), dtype=account_dtype).fillna("") # sheet_name="확정,미확정",
# # # 타입 등 후처리
# # df.replace({pd.NaT: ""}, inplace=True)
# # df["상환후잔액"] = df["상환후잔액"].replace("",0).astype(np.int64)
# df.drop(df[df.채무자명=="예수금"].index, inplace=True)
# df.reset_index(drop=True, inplace=True)
# # 저장 : 파일명 수정
# df.to_pickle(join(wd, basedate, "pkl", filename +".pkl"))
# deposit.to_pickle(join(wd, basedate, "pkl", files[6]+".pkl"))

In [ ]:
### 옛날 거 할 때 임시 ####################################
# credit.rename(columns={"OPB":"반영OPB", "상환방식":"상환구분"}, inplace=True)
# rehabilitation.rename(columns={"인가/미인가":"인가미인가"}, inplace=True)
# credit.rename(columns={"접수일자":"접수일"}, inplace=True)
# credit.rename(columns={"신청인":"신청인명"}, inplace=True)

# rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"].copy()
# rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"].copy()
# credit_d = credit[credit["채무구분"]=="주채무"].copy()
# credit_grt = credit[credit["채무구분"]=="보증채무"].copy()

In [9]:
# 개회, 신복 반영OPB 빈값 있는지, int64인지 확인 : 반영opb에는 빈값이 있으면 안 됨. object로 나오면 빈값이 있는 거고, 담당자에게 파일 수정 요청
print(len(rehabilitation[rehabilitation.반영OPB==""]))
print(len(credit[credit.반영OPB==""]))
print(rehabilitation.반영OPB.dtype, credit.반영OPB.dtype)

0
0
int64 int64


In [65]:
# # pkl 저장
if not os.path.exists(join(wd, basedate, "pkl")) : 
    os.mkdir(join(wd, basedate, "pkl"))

debt.to_pickle(join(wd, basedate, "pkl", files[0]+".pkl"))
account.to_pickle(join(wd, basedate, "pkl", files[1]+".pkl"))
grt.to_pickle(join(wd, basedate, "pkl", files[2]+".pkl"))
rehabilitation.to_pickle(join(wd, basedate, "pkl", files[3]+".pkl"))
credit.to_pickle(join(wd, basedate, "pkl", files[4]+".pkl"))
event.to_pickle(join(wd, basedate, "pkl", files[5]+".pkl"))
if len(files) >= 7 : 
    deposit.to_pickle(join(wd, basedate, "pkl", files[6]+".pkl"))

### 파일 수정
- 시효(계좌비고에 "채권시효만료") : 계산된 시효는 남은 경우 있고, 대부분 종료건. 파일 수정 없이 시효 체크시에만 사용

#### 파산
- 보증인 사건번호에는 현재결과가 없으므로 법조치에 등록되어 있어야만 한다.
- 차주는 현재결과가 있더라도, 당사포함여부를 확실하게 하려면 법조치에 등록되어 있어야만 한다.
    - 일단은 채무자 사건에 파산 사건 있다면, 당사 포함으로 간주한다.

##### 차주

In [ ]:
파산사건등록건 = debt.query('파산사건번호.str.contains("하")')[["채무자키","주민번호인","채무상태", "담당자", "파산사건번호","파산사건결과"]].copy() # 파산사건결과 없는 건은 차주에 대한 것이 아니거나 법원 또는 사건번호에 오류 있거나. (사전에 체크하고, 최신화 해두어야)
pattern = re.compile('[가-힣]+')

# debt 채무상태 수정
"""
todo
회생사건, 관리자사건 파산사건으로 이전, 사건번호에 "/누락"포함건 법조치 등록(솔림완료)
일반법원 -> 회생법원
하단 -> 하면, 최신화 버튼 눌러주기
todo끝
또한 파산사건에 파산 아닌 다른 것이 기록되어 있는 경우도 있다. .str.contains("하") 필요 
하단 : (신청)취하, 기각 / 파산선고, 파산선고및동시폐지, 종결, 폐지(하면으로 넘어감)
하면 : (신청)취하, 기각 / 불허가 / 인용 ** 채무자의 : 채무자의 사망으로 종결
파산(진행확정) : 선고, 종결, 폐지
파산(기각) : 기각, 취하, 불허가, 채무자의
파산(면책) : 인용
"""

# 차주 파산사건번호와 법조치조회로 bankruptcy_d, bankruptcy_grt 만든다. > 개회,신복과 충돌여부 확인 및 종결건 아닌 경우 raw_data 새채무상태에 반영한다.
bankruptcy_d = pd.DataFrame(None, columns=["채무자키","주민등록번호","진행구분","접수일","종료일", "종국결과"])

# 채무자
for i, v in 파산사건등록건.iterrows() : 
    # 사건결과 없는 건 : 법원,사건번호 오류이거나 차주가 아니거나, 진행 초기 -> 채무상태값을 따르되, bankruptcy_d에는 추가하지 않는다.
    if v.파산사건결과 == "" :
        pass
    # 기각아니고 미종결건 : 현재결과로 채무상태 업데이트
    elif re.search("취하|불허가|기각|채무자의",v.파산사건결과)==None and re.search(comp_closing["종료"], v.채무상태+v.담당자)==None :
        파산사건등록건.loc[i, "채무상태"] = "파산("+ pattern.search(v.파산사건결과)[0]+ ")"
        
        # 파산사건결과.add(파산사건등록건.loc[i, "채무상태"])
        # 결과 : '채무자의'  (채무자의 사망으로 종료) 

# 확인 ###########
merge = pd.merge(파산사건등록건, debt[["채무자키", "채무상태", "파산사건결과"]], how='outer', on='채무자키', suffixes=('_new', '_ori'))
# merge[merge.파산사건결과 != ""]
# merge[merge.채무상태_ori != merge.채무상태_new]

In [ ]:
debt.파산사건결과.str.extract('([가-힣]+)').value_counts()

In [ ]:
# 확인 후 이상없으면 저장  # 이상없이 잘 되는 거 확인 함 # 파일 저장은 하지 말자.
debt["채무상태"] = np.where(debt["채무상태"].str.contains("누락"), debt["채무상태"], temp["채무상태"])

#### 개회, 신복 충돌 체크 
- 종료, 비정상건과 충돌해도 삭제 x. 
  - 풀별 파일 만들때 새채무상태가 조정인 채무자키만 참조하면 되고, 매각후에도 계속 봐야하므로 기준데이터에 남긴다.
  - 비정상으로 분류했지만, 부활이거나 상태값이 잘못된 건의 경우 조정채권이 될 수도 있음. 환매 없앨려면 이건 따로 리스트 만들어 내부 정리하는 게 매각리스트로 뽑는 거 보다 낫겠다.

In [38]:
# [check]개회와 신복 채무자키 중복 : 차주, 보증인별
# 신복실효신청완료 = ["20452103", "20452608","20463594","20463620","20464387"] >>> 실효 됐는지 체크
rehabilitation_cols = ["채무자키","계좌키","분납키","분납자관계","분납자성명인","인가미인가","반영OPB", "접수일"]
credit_cols = ["채무자키","계좌키","접수번호","채무구분","신청인명","진행구분","반영OPB","접수일"]
account_cols = ["채무자키", "계좌키", "채무자명", "채무상태", "담당자"]

# 차주
조정중복_차주 = pd.merge(rehabilitation_d[rehabilitation_cols], credit_d[credit_cols], on=["채무자키","계좌키"], how="inner", suffixes=('_개회', '_신복'))
display("차주", 조정중복_차주)

# 보증인
조정중복_보증인 = pd.merge(rehabilitation_grt[rehabilitation_cols].rename(columns={"분납자성명인":"신청인명"}), credit_grt[credit_cols], on=["채무자키","계좌키", "신청인명"], how="inner", suffixes=('_개회', '_신복'))
display("보증인", 조정중복_보증인)

##### 회생 vs 파산 차주
# 회생파산중복_차주 = pd.merge(rehabilitation_d[rehabilitation_cols], account[])


##### 신복에 실효요청할 수 있도록



# 중복 없으면 아래 두개 셀 실행 안 해도 됨

'차주'

,채무자키,계좌키,분납키,분납자관계,분납자성명인,인가미인가,반영OPB_개회,접수일_개회,접수번호,채무구분,신청인명,진행구분,반영OPB_신복,접수일_신복


'보증인'

,채무자키,계좌키,분납키,분납자관계,신청인명,인가미인가,반영OPB_개회,접수일_개회,접수번호,채무구분,진행구분,반영OPB_신복,접수일_신복


In [11]:
# 접수일자에 따라서 신복, 개회 하나만 남기기 
print(f'삭제전 개회 : {len(rehabilitation)}, 삭제전 신복 : {len(credit)}')
if len(조정중복_차주)>0 :
    개회index, 신복index = [], []
    for i, v in 조정중복_차주.iterrows() :
        if datetime.strptime(v.접수일_개회, "%Y-%m-%d") > datetime.strptime(v.접수일_신복, "%Y-%m-%d") : # 개회접수일 나중인 경우, 신복에서 제거
            신복index.append(credit[(credit.채무자키 == v.채무자키) & (credit.계좌키 == v.계좌키)].index.values[0])
        else : 개회index.append(rehabilitation[(rehabilitation.채무자키 == v.채무자키) & (rehabilitation.계좌키 == v.계좌키)].index.values[0])
    print(신복index)
    print(개회index)
    if len(신복index)>0 : credit = credit[~credit.index.isin(신복index)]
    if len(개회index)>0 : rehabilitation = rehabilitation[~rehabilitation.index.isin(개회index)]

if len(조정중복_보증인) > 0 :
    개회index, 신복index = [], []
    for i, v in 조정중복_보증인.iterrows() :
        if datetime.strptime(v.접수일_개회, "%Y-%m-%d") > datetime.strptime(v.접수일_신복, "%Y-%m-%d") : # 개회접수일 나중인 경우, 신복에서 제거
            신복index.append(credit[(credit.채무자키 == v.채무자키) & (credit.계좌키 == v.계좌키) & (credit.신청인명 == v.신청인명)].index.values[0])
        else : 개회index.append(rehabilitation[(rehabilitation.채무자키 == v.채무자키) & (rehabilitation.계좌키 == v.계좌키) & (rehabilitation.분납자성명인 == v.신청인명)].index.values[0])
    print(신복index)
    print(개회index)
    if len(신복index)>0 : credit = credit[~credit.index.isin(신복index)]
    if len(개회index)>0 : rehabilitation = rehabilitation[~rehabilitation.index.isin(개회index)]
print(f'삭제후 개회 : {len(rehabilitation)}, 삭제후 신복 : {len(credit)}')

삭제전 개회 : 3523, 삭제전 신복 : 12179
[1049, 694, 4159]
[]
삭제후 개회 : 3523, 삭제후 신복 : 12176


In [39]:
# 확정된 조정채권 차주, 보증인 저장하기. 종결건에도 따로 문제발생해서 처리했다면 이 셀 재 실행 해야..
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"].copy()
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"].copy()
credit_d = credit[credit["채무구분"]=="주채무"].copy()
credit_grt = credit[credit["채무구분"]=="보증채무"].copy()

#### grt, 새보증인상태 추가
- 새보증인상태 들어간 파일을 읽었다면 PASS

In [19]:
# 개회 >> 차주 : 계좌키, 보증인 : [계좌키 + 보증인이름]이 pk. 
# 신복 >> 보증인키 있으므로 이걸로. 누락계좌는 정상으로 나옴
# 개회 누락채권(일부계좌누락, 일부 채무자키(이런 경우가 있을까?)) 문제는 어떻게? : 일단 
check_grt_index = []
# 보증인 상태변경
if "새보증인상태" in grt.columns : grt["새보증인상태"] = ""
else : grt.insert(8,"새보증인상태","")

# 종료, 면책, 종료권고 채우기 (보증인의 경우 폐업-> 종료권고)
cond1 = grt.종결일 != ""
cond2 = grt.보증인상태.str.contains(comp_closing["종료"])
cond3 = grt.보증인상태.str.contains(comp_closing["시효"]) # 보증인 상태값에도 시효가 있네..check
cond4 = grt.보증인상태.str.contains(comp_closing["면책"])
cond5 = grt.보증인상태.str.contains(comp_closing["불가"])
cond6 = grt.보증인상태.str.contains("폐업")
condlist_grt = [cond1, cond2, cond3, cond4, cond5, cond6, True]
valuelist_grt = ["종료", "종료", "시효", "면책", "불가", "불가", ""]
grt["새보증인상태"] = np.select(condlist_grt, valuelist_grt )

# '채무자키'열과 '보증인성명인'열을 기준으로 비교하여 값 변경
for i in tqdm(range(len(grt))):  
        # 개인회생
        temp = rehabilitation_grt[(rehabilitation_grt['채무자키'] == grt.at[i, '채무자키']) & (rehabilitation_grt['분납자성명인'] == grt.at[i, '보증인성명인'])]
        if len(temp) > 0: # 개인회생 중
            if grt.at[i, '새보증인상태'] != "" :# 보증인 개인회생 중인데 보증인상태는 비정상건으로 되어 있는 경우
                check_grt_index.append(i)
            
            # 새보증인상태는 조정중인 것으로 
            grt.at[i, '새보증인상태'] = "개인회생(확정)" if temp['인가미인가'].values[0] == "인가" else "개인회생(진행중)"
                
        else : 
            # 신용회복
            temp = credit_grt[(credit_grt['보증인키'] == grt.at[i, '보증인키'])]
            if len(temp) > 0:
                if grt.at[i, '새보증인상태'] != "" : # 보증인 신용회복 중인데 보증인상태는 비정상건으로 되어 있는 경우
                    check_grt_index.append(i)
                # 새보증인상태는 조정중인 것으로 
                grt.at[i, '새보증인상태'] = "신용회복("+ temp['상환구분'].values[0]+")" if temp['진행구분'].values[0] == "확정" else "신용회복(진행중)"
            else :
                # 새보증인상태가 종결, 비정상, 조정이 아닌 경우 => 무담보
                if grt.at[i, '새보증인상태'] == "" :
                    grt.at[i, '새보증인상태'] = '정상'
    
# 보증인상태가 종료,시효,면책,불가로 되어있는데 조정중인 경우 check
if len(check_grt_index) == 0 :
    print("보증인상태 충돌건 없음")
else : 
    print(len(check_grt_index),"개의 보증인상태 충돌")
    grt.iloc[check_grt_index].to_excel(join(wd, basedate, "[check]보증인상태충돌_조정vs비정상,종결.xlsx"))

100%|██████████| 10406/10406 [00:03<00:00, 3280.20it/s]

보증인상태 충돌건 없음


In [70]:
# 저장 : 보증인 수정할 것 엑셀 저장
# 수정해야 함 : grt[grt.새보증인상태 != grt.보증인상태].loc[:,["채무자키", "계좌키","보증인키","매각사구분","보증인성명인","주민번호인","계좌번호","보증구분","새보증인상태","보증인상태","보증금액","보증잔액","보증한도액"]].to_excel(join(wd, basedate, "9.보증인상태변경할것.xlsx"), index=False)
# # 새보증인상태 추가한 파일 저장
grt.to_pickle(join(wd, basedate, "pkl", files[2]+"_새보증인상태.pkl"))
grt.to_excel(join(wd, basedate, files[2]+"_새보증인상태.xlsx"), index=False)

### 전체 원데이터 만들기(종합)

In [50]:
# 약정파일 읽기 : 분납상태 진행 & 분납연체일 90일 이하
file_list = os.listdir(join(wd, basedate))
fn_약정 = [file for file in file_list if file.startswith("약정분납새창")][0]
        
약정 = pd.read_excel(join(wd, basedate, fn_약정), dtype = 약정_dtype).fillna("")
약정중 = 약정.query('분납상태=="진행" and 분납연체일 <= 90')[["채무자키", "분납연체일"]]

# account와 병합
account = account.merge(약정중, on="채무자키", how='left')

In [51]:
# raw_data 정의
if True :
    raw_data = pd.DataFrame(None)
    raw_data["채무자키"] = account.채무자키
    raw_data["계좌키"] = account.계좌키
    raw_data["타채무자키"] = account.타채무자키

    raw_data["채권구분"] = account.채권구분 ##
    raw_data["채권상태"] = "" ##
    raw_data["새채무상태"] = "" ##
    raw_data["채무상태"] = account.채무상태 ##
    raw_data["담당자"] = account.담당자 ##
    raw_data["성명"] = account.채무자명 #
    raw_data["주민등록번호"] = account.채무자주민번호 # #.apply(lambda x : x[:8])
    raw_data["계좌번호"] = account.계좌번호
    raw_data["원채권사"] = account.원채권사
    raw_data["매각사구분"] = account.매각사구분
    raw_data["계정과목"] = account.계정과목
    raw_data["은행대출원금"] = account.은행대출원금 # 대주단 양식에 있는 경우 있어서 삽입, 자산정리시에는 제외
    ###############################################
    # 매입최초금액 수정 전
    raw_data["최초원금"] = np.where(account.계좌조정일=="", account.최초원금, account.매입최초원금)
    raw_data["최초법비용"] = np.where(account.계좌조정일=="", account.최초법비용, account.매입최초법비용)
    raw_data["최초미수이자"] = np.where(account.계좌조정일=="", account.최초미수이자, account.매입최초미수이자)
    # 매입최초금액 수정 후
    # raw_data["최초원금"] = account.매입최초원금
    # raw_data["최초법비용"] = account.매입최초법비용  
    # raw_data["최초미수이자"] = account.매입최초미수이자  
    ###############################################
    raw_data["연체이자율"] = account.연체이자율 # 240118 최초연체이자율 -> 연체이자율. 최초는 소수점아래 3자리 이상 & 이자율 없어서 추가한것은 연체이자율에 추가중
    raw_data["현재원금"] = account.현재원금
    raw_data["현재법비용"] = account.현재법비용
    raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
    raw_data["매입OPB"] = account.OPB # 추가 2024-07~
    raw_data["현재미수이자"] = account.현재미수이자
    raw_data["현재연체이자"] = account.현재연체이자
    raw_data["현재합계"] = account.현재합계
    raw_data["회수합계"] = account.회수합계 ## 부채증명서 등 입금기타로 잡힌 것도 포함, 발송에선 제외함
    raw_data["최초대출일"] = account.최초대출일
    raw_data["대출만기일"] = account.대출만기일
    raw_data["최초연체일"] = account.최초연체일 # 없는 경우도 있음 없으면 대출만기일로 대체해야.
    raw_data["매입일"] = account.매입일
    # 시효관련 추가
    raw_data["시효실사결과"] = account.시효완성여부
    raw_data['시효완성월'] = np.where((account['시효중단여부'] == 'Y'), '중단', account['시효완성월'])
    raw_data["시효사유"] = account.시효연장사유 # 시효연장사유 빈칸인 경우, 최초완성일메모(거의), 최초시효완성일, 시효중단여부 모두 빈칸임
    raw_data["시효완성일"] = account.시효완성일
    raw_data["종결일"] = account.종결일
    raw_data["메모"] =account.메모
    raw_data["원인서류유무"] =account.원인서류확인여부

    # 최종 거래일 
    temp_a = account.최종거래일
    temp_b = account.최초최종거래일
    temp_c = account.계좌조정일
    raw_data["최종거래일"] = np.where(temp_a >= temp_b, np.where(temp_a>=temp_c, temp_a, temp_c), np.where(temp_b >= temp_c, temp_b, temp_c))

    # 내부약정(자체채무조정)
    raw_data["분납연체일"] = account.분납연체일

    # 채무자조회에서 불러올 항목 , 초본주소에 초본>자택>직장>기타 주소순으로 입력되어 있음
    debt_temp = debt[["채무자키", "개인법인", "연령", "초본우편번호", "초본주소", "관리자비고"]].rename(columns={"관리자비고":"새채무자키", "초본우편번호":"우편번호", "초본주소":"주소"})
    raw_data = pd.merge(raw_data, debt_temp, on="채무자키", how="left")

    # grt에서 각 계좌별 (새)보증인상태 불러오기
    names = grt.groupby('계좌키')['보증인성명인'].apply(lambda x: ','.join(x)) # 고유값만 남기려면 join의 매개변수로 x.unique()를 사용, 계좌에 동일보증인이 둘일수는 없으므로 여기서는 할 필요없음
    newstatus = grt.groupby('계좌키')['새보증인상태'].apply(lambda x: ','.join(x))
    status = grt.groupby('계좌키')['보증인상태'].apply(lambda x: ','.join(x))

    raw_data = raw_data.merge(names, how='left', on='계좌키')
    raw_data = raw_data.merge(newstatus, how='left', on='계좌키')
    raw_data = raw_data.merge(status, how='left', on='계좌키')

    raw_data["보증인성명인"] = raw_data["보증인성명인"].fillna('')
    raw_data["새보증인상태"] = raw_data["새보증인상태"].fillna('')
    raw_data["보증인상태"] = raw_data["보증인상태"].fillna('')

    # 추심가능여부 : 추심가능한 채무자가 한명이라도 있는 경우
    # 가능(정상,누락,폐지,실효) 불가(면책,파산,개회,신복,종결요청,불가) * 불가건은 제외했음에 유의
    raw_data["추심가능여부"] = ""

In [52]:
# 일부계좌제외 관리
# 계좌메모에 '신용회복 제외채권'등록되어 있지 않거나, 전계좌가 실효된 경우 여기서 계좌메모 수정하고, 전산 수정파일도 만들어주자
                            # left : 이미 반영                                        #right : 반영할것
df = pd.merge(raw_data.query('메모.str.contains("신용회복 제외")')[["계좌키","채무자키"]], credit_excluded[["계좌키"]], on='계좌키', how='outer', indicator=True)
print(df._merge.value_counts())

# raw_data 수정
raw_data["메모"] = np.where(raw_data.계좌키.isin(df.query('_merge=="right_only"').계좌키), "신용회복 제외", raw_data["메모"])
raw_data["메모"] = np.where(raw_data.계좌키.isin(df.query('_merge=="left_only"').계좌키), "", raw_data["메모"])

# 전산 수정파일
# 채무조정제외건 발생시 일반 메모에는 등록하고 있으므로 계좌메모만 반영하면 된다
# 종료건은 제외하기
df = df[~df.계좌키.isin(raw_data.query('채권상태!="종료"').계좌키.values.tolist())]
df = df.query('_merge != "both"') # 수정할 필요 없는 경우

df["메모"] = np.where(df._merge=="right_only", "신용회복 제외", "NULL")
if len(df) > 0 :
    df[["계좌키","채무자키","메모"]].to_excel(join(wd, basedate, "[update]신용회복 제외"+basedate+".xlsx"), index=False)
else :
    display(df)

_merge
both          4
left_only     1
right_only    1
Name: count, dtype: int64


,계좌키,채무자키,_merge,메모


In [53]:
# 새채무자키사용 : 보증인 없는 계좌도 보증인 있는 계좌로 인식
# 보증인에 새채무자키 추가

# 금감원용으로는 False
#############################################
새채무자키사용 = True  # True / False
#############################################


if 새채무자키사용 :
    grt_새채무자키 = grt.merge(debt_temp[["채무자키", "새채무자키"]], on='채무자키', how='left')
    newstatus_새채무자키 = grt_새채무자키.groupby('새채무자키')['새보증인상태'].apply(lambda x: ','.join(x.unique())) # 계좌의 보증인에 대한 것이 아니라 고유값만 남김
    newstatus_새채무자키.name = "새보증인상태by새채무자키"

    raw_data["새보증인상태by새채무자키"] = raw_data.merge(newstatus_새채무자키, how='left', on='새채무자키')['새보증인상태by새채무자키'].fillna('')
else : 
    raw_data["새보증인상태by새채무자키"] = ""

#### 채권상태, 새채무상태 작성
- 시효완성여부(시효실사결과) : 시효완성/부활/미완성(?) 실사 결과는 계좌의 '시효완성여부'에 적고, 별도의 칼럼으로 표시한다.
- 채권상태 : 종료 -> 비정상(종료처리할것,보증인정상,시효,가지급계좌,공채권) -> 정상(무담보, 개회, 신복)
  - rawdata에서 채권상태를 불러와서 자산정리 하므로 기준데이터에 종료건 있어도 괜찮다.
- 새채무상태 : 채권상태 정상이면 개인, 법인, 개인회생(진행중), 개인회생(확정), 신용회복(개인), 신용회복(프리), 신용회복(진행중)

In [54]:
# 파산기준데이터 만들때까지는 빈df로
bankruptcy_d = pd.DataFrame(None, columns=["채무자키"])

In [55]:
############################
# 시효는 채무상태, 담당자에서 제거하는 게 목표. 시효실사결과에 시효는 적고, 매각 등 작업에서 엑셀파일을 통해 활용하면 된다.
# 변하면 안 되므로 완성과 부활만 적어준다. (새로운 정보가 발견되어 실사결과가 달라진 경우, 전산 바로 수정해줘야)
# 비정상 : 종료처리요망,보증인정상,시효,가지급계좌,공채권
# 보증인 : 보증인 등록 안 되어있는 건이 많으니 동일 차주에 대한 보증인 상태 전체를 살핀다.
# comp_closing = {
#     "종료" : r"종결(?!요청)|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료",
#     "종결요청" : r"종결요청|매각예정|환매예정",
#     "시효" : r"시효|완성|만료", # 부활 제외함 231222
#     "면책" : r"면책|면탈",  
#     "불가" : r"추심제외|상각|포기|파산(?!\(기각\))|사망(?!\(상속승인\))", # 상각, 제외, 포기는 이미 보증인까지 고려한 결과일 수 있음
#     "공채권" : r"공채권"
#     }
############################
채권상태, 새채무상태 = [], []
종료 = re.compile(comp_closing["종료"])
종결요청 = re.compile(comp_closing["종결요청"])
시효 = re.compile(comp_closing["시효"])
상각 = re.compile(comp_closing["상각"])
면책 = re.compile(comp_closing["면책"])
불가 = re.compile(comp_closing["불가"])
공채권 = re.compile(comp_closing["공채권"])

for i, v in tqdm(raw_data[["채무상태", "담당자", "종결일", "메모", "새보증인상태", "채무자키", "계좌키", "개인법인", "새보증인상태by새채무자키"]].iterrows(), total = len(raw_data)) :
    # 메모는 채무상태를 계좌별로 적용한 것이므로 state로 처리하면 되며, 우선순위를 주기위해 state의 젤 앞에 배치.
    state = v.메모 + v.채무상태 + v.담당자
    개회temp = rehabilitation_d[rehabilitation_d['계좌키'] == v.계좌키]
    신복temp = credit_d[credit_d['계좌키'] == v.계좌키]
    파산temp = bankruptcy_d[bankruptcy_d['채무자키'] == v.채무자키] # 채무자키에 유의.. 새채무자키로 해야하나?
    
    # 1.종료
    if (v.종결일 != "") or 종료.search(state) : 
        채권상태.append("종료")
        새채무상태.append("종료")
        # 메모를 state로 합쳐서 처리하기 전 코드 ---------------s
        # elif v.메모 != "" : # 현재 메모에는 '종료'만 있으나 "비정상"도 기입할 수 있으므로 별도 처리함.
        #     새채무상태.append(v.메모)
        #     if v.메모 == "종료" : 채권상태.append("종료")
        #     else : 채권상태.append("비정상")
        # 메모를 state로 합쳐서 처리하기 전 코드 ---------------e
    # 2.비정상(종결예정) - 종료처리요망
    elif 종결요청.search(state) : # 채무상태나 담당자가 '종결요청'
        채권상태.append("비정상")
        새채무상태.append("종료처리요망")
    
    # 3. 개회, 신복 - 현재 조정중이라면 기타비정상보다는 먼저 처리해야
    # 통합 채권은 각 기준파일에서 처리함(조정채권으로 분류) : 신복:신용회복키와 계좌진행상태에 표기, 개회 : 분납메모에 표시
    elif len(개회temp) > 0:
        채권상태.append("정상")
        새채무상태.append("개인회생(확정)" if 개회temp['인가미인가'].values[0] == "인가" else "개인회생(진행중)")
                    
    elif len(신복temp) > 0:
        채권상태.append("정상")
        새채무상태.append("신용회복("+ 신복temp['상환구분'].values[0]+")" if 신복temp['진행구분'].values[0] == "확정" else "신용회복(진행중)")
                
    # 4. 비정상(추심가능)
    # 추심불가건이나 혹시나 잘못 바꿨을까봐 조정채권 다음에 둔다. 채권 전체에 대해 실사한 것이므로 다른 불가건보다 먼저 해야 함
    elif 상각.search(state):
        채권상태.append("비정상")
        새채무상태.append("상각")
        
    # 4-1. 시효
    elif 시효.search(state):
        채권상태.append("비정상")
        새채무상태.append("시효")
    # 4-2 - 차주 불가건 : 면책,파산,사망,청산,폐소,폐업,포기,제외 - 보증인에 따라서
    elif 면책.search(state) or 불가.search(state) or len(파산temp) > 0 :
        채권상태.append("비정상")
        if 새채무자키사용 : target = v.새보증인상태by새채무자키
        else : target = v.새보증인상태
        # 새채무자키 기준 보증인 중 정상상태 있으면 ######################
        if re.search('개인회생|신용회복|정상', target) :  # cf) v.새보증인상태 , 새보증인상태의 정상은 면책불가를 제외한 것이므로 누락, 약정 이런 거 모두 포함되어 있음
            새채무상태.append("보증인정상")
        else :
            새채무상태.append("종료처리요망")
    # 4-3. 공채권 : 계좌 통합신고 등의 사유로 0원처리해야 하는 것(cf.개별신고했으나 반영OPB가 0원인 것은 조정채권이다.-분납메모의 공채권)
    elif 공채권.search(state) :
        채권상태.append("비정상")
        새채무상태.append("공채권")
    # 정상 : 법인 별도 처리 
    else :
        temp채권상태 = "정상"
        if re.search("개인", v.개인법인) : # 대성 개인사업자때문에 변경 240103 cf)if v.개인법인 == "개인"
            새채무상태.append("개인")
        # 정상 - 법인인 경우 새채무자키 기준 보증인 상태 재확인 ######################
        else :
            if 새채무자키사용 : target = v.새보증인상태by새채무자키
            else : target = v.새보증인상태
            if re.search('개인회생|신용회복|정상', target) : 
                새채무상태.append("법인")
            else : # 정상보증인 없음 : 채권상태 비정상
                새채무상태.append("종료처리요망")
                temp채권상태 = "비정상"
        채권상태.append(temp채권상태)

# 작성
raw_data["채권상태"] = 채권상태
raw_data["새채무상태"] = 새채무상태


100%|██████████| 78406/78406 [01:08<00:00, 1137.90it/s]


In [56]:
# 가지급금 계좌 처리 : 클렌징 작업대상으로 분류(다채무상태로 걸리겠지, 경우의 수를 전부 확인하고 처리 방법을 만들어야 한다.) - 차주가 법인인 경우가 많아 주로 종결요청으로 분류될거다.
# 예상되는 경우 : 1. [비정상-종결요청]"개회 통합신고, 신복 통합신고, 타계좌는 보증인 정상인데" 당 계좌는 신고가 안 되거나 보증인 등록 안 돼서 종결요청(비정상)으로 분류
#                2. [정상-개인/법인]차주가 정상이라 보증인 등록 안 되어 있어도 개인으로 분류(이 경우 통합신고가 되어있다면 '공채권'을 계좌메모에 적어서 opb 0원처리 해줘야함)
#                3. [정상-조정]개회, 신복에 개별계좌로 신고되었으나 조정후금액이 0원 : 이 경우 분납메모에 '공채권'을 적어서 opb가 0원인 이유를 알 수 있게 해줘야 함.
#                4. [모든계좌-종결요청] 모든 채무자가 종결사유에 해당되는 경우는 빨리 종결처리해서 혼란을 없애주어야 함
# ====> 해당계좌가 비정상으로 분류되었는데, 동일 "새채무자키"를 사용하는 계좌에 정상(보증인정상)건이 있는 경우, 새채무상태를 "가지급금"으로 해주자.

# 살려야할 가지급계좌 : 해당 계좌는 종료처리요망이라도, 동일새채무자키를 쓰는 계좌 중 채권상태-정상(개인,법인,조정)/새채무상태-보증인정상|시효건이 있는 경우
# 이런 경우는 가지급계좌라 보증인이 등록되어있지 않은 것이므로
가지급계좌 = raw_data.query('(계정과목.str.contains("가지급") or 메모.str.contains("가지급")) and 새채무상태=="종료처리요망"') 
print("가지급계좌 수 :", len(가지급계좌))

상태변경건 = []
for i, v in tqdm(가지급계좌.iterrows(), total=len(가지급계좌)) : 
    동일차주 = raw_data.query('새채무자키 == @v.새채무자키')
    채권상태들 = 동일차주.채권상태.values
    새채무상태들 = 동일차주.새채무상태.values
    
    # 동일차주 계좌 중 정상 건 또는 보증인정상|시효건이 있는 경우의 가지급계좌는 새채무상태를 종료처리요망이 아니라 가지급계좌로 표시해주자
    # 종결이나 정상(조정채권으로 분리신고된 경우 포함)은 가지급계좌로 표시할 필요가 없음
    if ("정상" in 채권상태들) or ("보증인정상" in 새채무상태들) or ("시효" in 새채무상태들): 
        raw_data.loc[i,"새채무상태"] = '가지급계좌'
        상태변경건.append(i)
if len(상태변경건) > 0 :
    print('새채무상태 가지급으로 변경건')
    display(raw_data.iloc[[1,2,3], :10])
else : 
    print('새채무상태 변경건 없음')

가지급계좌 수 : 30


100%|██████████| 30/30 [00:00<00:00, 364.80it/s]

새채무상태 변경건 없음


In [ ]:
# [check] 종결처리 해야 하는 건 저장하기 - 코드 수정해야
# cond1 = raw_data.종결일 == ""
# cond2 = (raw_data.채무상태.str.contains(comp_closing["면책"]) | raw_data.담당자.str.contains(comp_closing["면책"])) & ~(raw_data.새보증인상태.str.contains("정상|개인회생|신용회복"))
# raw_data[cond1 & cond2].to_excel(join(wd, basedate, "[check]종결처리할것-완적면책.xlsx"))
# [ckeck]종결처리해야하는 계좌 출력
# if len(종결처리안된종료건)>0 :
#     # print(종결처리안된종료건)
#     pd.DataFrame(종결처리안된종료건).to_excel(join(wd, basedate, "[check]종결처리안된종료건.xlsx"), index=False)
# else : print("이상무")

In [ ]:
# [ckeck]계좌별 채무상태 수정할 것 엑셀 저장
# 범주가 다르게 적용되어서 현재는 의미가 없다. 보완이 필요
# raw_data[raw_data.새채무상태 != raw_data.채무상태].loc[:,["채무자키", "계좌키","매각사구분","새채무상태","채무상태"]].to_excel(join(wd, basedate, "[check]계좌별채무상태달라진것.xlsx"), index=False)

#### 기준일 이후 입금 등록 및 수정건 반영
- t일 계좌에 조정입금과 일반입금 반영한 opb만 만들면 됨.(일반입금 없어도 해야함)
- 신용회복기준데이터파일은 신용회복 탭에서 미리했음
- 개인회생기준데이터파일은 기준일 입금 후 전산 다운받아 기준데이터 만들기 때문에 안해도 됨

#### 입금등록 pass

In [ ]:
# 입금제외 : 입금기타에 금액잡힌 것(현재금액에 영향을 주지 않는 것), 입금기간에 값이 있는 것 : 입금 취소, 환매대금 등
print(f'전체입금건수 : {len(deposit)}')
deposit = deposit[(deposit.입금기타 == 0) & (deposit.입금기간 =="")] # 신복 실효건은 여기서 걸러졌다.(계좌조회새창을 수정했으므로 걸러지는 게 맞다.)
deposit = deposit[deposit.입금구분 != "부채증명서"]
print(f'제외건 빼고(부채증명서,입금기간있는거) 남은 건수 : {len(deposit)}')

# 등록일자, 수정일자 yyyy-mm-dd 형태로
deposit['등록일자'] = deposit.등록일자.str[:10]
deposit['수정일자'] = deposit.등록일자.str[:10]

# 후일입금만 고르기 : 기준일 이내에 입금건 중 등록이 기준일(계좌다운일) 이후인 것 (애초에 입금내역을 등록일이 기준일 이후인 것으로 다운받으면 등록일자>@기준일 조건은 필요없으나 만일을 위해 지우지 않음)
후일입금 = deposit.query('등록일자 > @기준일 and 입금일 <= @기준일').copy() # [기준일 이전 등록, 기준일 이후 수정]때문에 원래는 수정일자로 했으나, 이 경우는 중복입금처리하기 보다는 그냥 수정반영을 안 하는게 낫겠다.
# 기준일 신복 실효건 : 입금기간이 원상회복전이므로 위에서 걸러짐
print(f'계좌 미반영 후일입금 건수 : {len(후일입금)}')

# 후입금의 경우 후입금보다 입금일이 나중이고 직접입력인 입금건이 있다면 해당건의 현재금액이나 회수금액은 수정이 되지 않기 때문에 입금창의 현재금액을 가지고 계산을 돌리면 안 된다.
# 따라서 다건이건 단건이건 계좌에 있는 현재금액에서 후일건을 모두 처리해준다.
# 단, 후일건중 기준일 이전의 입금원금이 있는 경우 : 날짜차이만큼 계좌의 현제연체이자에서 차감
print("기준일 이전의 원금변제 있는 후입금건")
display(후일입금.query('입금구분 != "신용회복" and 입금구분 != "개인회생" and 입금원금>0 and 입금일<@기준일'))

# 금액적으로나 시효측면에서나 정렬은 할 필요없음.(기존시효와 비교하므로)

In [ ]:
# 확인하고 싶을 때만 출력. 24529
# 후일입금.to_excel(join(wd, basedate, "후일입금.xlsx"), index=False)

In [ ]:
후일입금["날짜차이"] = 후일입금["입금일"].apply(날짜차이, 기준일=기준일)
# 입금원금분의 이자. 다건이든 단건이든 이걸 계좌연체이자에서 빼주면 됨
                                    # 연이자                                            #일할
후일입금["차감할연체이자"] = (후일입금["입금원금"] * 후일입금["적용이자율"] / 100 ) * (후일입금["날짜차이"] / totalday)
후일입금["차감할연체이자"] = 후일입금["차감할연체이자"].astype(int) # 소수점이하 버리기
# 연체이자 계산할 것 확인
후일입금.query('차감할연체이자 > 0')[["채무자키", "계좌키", "입금일", "날짜차이", "현재원금", "입금원금", "적용이자율", "차감할연체이자"]]

In [ ]:
# 원데이터 현재원금, 현재법비용, OPB, 현재미수이자, 현재연체이자, 현재합계, 회수합계, 시효완성일, 시효사유 수정
for i, v in tqdm(후일입금.iterrows(), total=len(후일입금)) : 
    ind = raw_data[raw_data.계좌키 == v.계좌키].index.values[0] # index까지만 하면 시리즈가 반환되어 loc했을 때도 시리즈가 반환된다.
   
    # 한계좌에 여러건의 후일입금이 있어도 
    raw_data.loc[ind, "현재원금"] -= v.입금원금
    raw_data.loc[ind, "현재법비용"] -= v.입금법비용
    raw_data.loc[ind, "현재미수이자"] -= v.입금미수이자
    raw_data.loc[ind, "현재연체이자"] -= (v.입금연체이자 + v.차감할연체이자)
    
    raw_data.loc[ind, "현재합계"] -=  v.입금합계 # 입금기타 있는 건 모두 제외했으므로 빼지 않아도 된다.
    raw_data.loc[ind, "회수합계"] += v.입금합계 # 전산시스템이 입금제외건(가수금)도 회수에는 합해지지만, 그게 오히려 잘못된 거니까..
    
    raw_data.loc[ind, "OPB"] = raw_data.loc[ind, "현재원금"] + raw_data.loc[ind, "현재법비용"]
    
    # 시효완성월은 아래 시효 작성하기에서 전체를 다시 하므로 여기서는 하지 않는다.
    새시효완성일 = str(int(v.입금일[:4]) + 5) + v.입금일[4:] # 전산은 -1일 하지만, 그게 틀린 것이므로 # 왜 연도에 +4를 하고 있었지? 240529 +5로 수정함
    if raw_data.loc[ind, "시효완성일"] < 새시효완성일 : # 문자대소비교 가능. 입금으로 시효가 연장된 경우
        raw_data.loc[ind, "시효완성일"] = 새시효완성일
        raw_data.loc[ind, "시효사유"] = "일반입금 입금일:"+새시효완성일
    else : pass # 원래 시효가 더 나중인 경우 
    
    # 신용회복은 기준일(말일)에 받은 계좌별 진행상황에 다음날 입금내역을 추가해서 기준데이터 작성했으므로 수정할 거 없음
    # 개회는 입금 처리 후 다운 받을 거니까 수정할 거 없음

#### 이자 빼기 : 컷off 이후에 계좌조회한 경우

In [58]:
# 계좌조회새창 파일에서 조회일 가져오기
조회일 = files[1].split("_")[1]
조회일 = 조회일[:4]+"-"+조회일[4:6]+"-"+조회일[6:]

# 조회일과 컷오프일 사이의 날짜차이
연체이자차감일 = abs(날짜차이(조회일,기준일))

차감전연체이자 = raw_data.현재연체이자.sum()
# 이자차감
raw_data["현재연체이자"] -= (raw_data["현재원금"] * raw_data["연체이자율"] / 100 ) * (연체이자차감일 / totalday)
차감후연체이자 = raw_data.현재연체이자.sum()
# 총 차감된 이자
print(f'차감된 이자 : {int(차감전연체이자) - int(차감후연체이자):,}') # 7월 하루 연체이자 3억

차감된 이자 : 613,855,033


#### 반영OPB 작성하기
- 비정상도 정상과 마찬가지로 무담보와 조정채권 OPB를 각각 반영한다.(내부용으로 봐야하니까)
- 종결은 모두 0으로 처리한다

In [60]:
print(len(raw_data))
raw_data = raw_data.drop(columns=raw_data.columns[raw_data.columns.str.contains("반영OPB")])

# 반영OPB 작성하기 - 차주가 조정인 경우만 반영OPB 수정
# 1. 조정채권 OPB 입력
개회OPB = rehabilitation_d.loc[:, ["계좌키", "반영OPB"]]
신복OPB = credit_d.loc[:, ["계좌키", "반영OPB"]]
신복OPB
# 개회OPB와 신복OPB 병합 (반영OPB_X, _Y가 되지않도록 ON = [])
merged_opb = 개회OPB.merge(신복OPB, on=['계좌키','반영OPB'], how='outer')
# raw_data와 병합하며 "반영OPB"칼럼 만들기
raw_data = raw_data.merge(merged_opb, on='계좌키', how='outer') # 혹시 뭔가 잘못된 게 있나 확인하기 위해 일부러 outer

# 2. 조정채권 아닌 것 OPB로 채우기
raw_data.loc[raw_data['반영OPB'].isnull(), '반영OPB'] = raw_data.loc[raw_data['반영OPB'].isnull(), "OPB"]

# 3. 0원 처리 : 공채권, 종료(매각, 종결 등 )건
raw_data["반영OPB"] = np.where((raw_data['채권상태'] == '종료') | (raw_data["새채무상태"] == "공채권"), 0, raw_data['반영OPB'])

# 자료형 : 정수
raw_data["반영OPB"] = raw_data["반영OPB"].astype(np.int64)
print(len(raw_data))

78406
78406


#### 시효 작성하기
- 법조치 시효가 정확하지 않을 수 있으므로 제한적으로 적용 : 무담보건에 대해, 시효만료 및 임박에 대해, 확장한 시효가 더 많은 경우만 수정
- 새채무자키로 통일, 차주-보증인구분 x
- 위에서는 시효를 가지고 뭘 한 것은 없음. 시효작성을 먼저하고, '채무상태-시효'처리를 나중에 하는 것으로 코드 수정해야 할 수도

In [6]:
# 바로 할 때는 건너띄기 #################################################################################################################################
작업경로=join(wd, basedate,"pkl")
path_raw_data_pkl = functions.키워드로파일명찾기(폴더=작업경로, 포함키워드="전체_원데이터")
path_event_pkl = functions.키워드로파일명찾기(폴더=작업경로, 포함키워드="법조치조회새창")

raw_data = pd.read_pickle(path_raw_data_pkl)
event = pd.read_pickle(path_event_pkl) 

# pkl 없을때
# raw_data = pd.read_excel(join(r"D:\3.자산\전산 dataset\솔림\230731", "2.솔림_전체_원데이터_230731"+".xlsx"), dtype=account_dtype, sheet_name="리스트").fillna("")
# event = pd.read_excel(join(wd, basedate, files[5]+ext), dtype=event_dtype).fillna("")

In [7]:
# 법조치에 새채무자키 입히기
event = pd.merge(event, raw_data.drop_duplicates('채무자키')[["채무자키","새채무자키"]] , on="채무자키", how='left')

In [8]:
# int() 에러 안 나게 빈문자열은 '0'으로
event['종료일'] = event['종료일'].apply(lambda x: '0' if x == '' else x)
event['종국결과일'] = event['종국결과일'].apply(lambda x: '0' if x == '' else x)
event['확정일'] = event['확정일'].apply(lambda x: '0' if x == '' else x)
event['접수일'] = event['접수일'].apply(lambda x: '0' if x == '' else x)

In [ ]:
# 집행권원만 모으기 
법조치구분조건 = r"본안소송|송달,확정증명원|수통부여|승계집행문|시효연장소송|재도부여|전자독촉|지급명령" # contains / ==
법조치세부조건 = r"본안소송|송달,?확정|(송달|확정)증명|수통부여|승계집행|판결문|시효연장|재도부여|전자독촉|지급명령" # contains

구분조건 = (event.법조치구분.str.contains(법조치구분조건)) | (event.법조치세부.str.contains(법조치세부조건))
사건번호조건 = event.사건번호.str.match(r"\d{4,4}(가[합단소]|나|다|머|차)") # match
종국결과조건 = event.종국결과.str.contains(r"결정|발급|승소|원고승|일부승|이행권고|화해|인용|지급명령|확정|항소기각|항소취하간주|조정(?!불)") # 패소가 아닌 경우 : ~event.종국결과.str.contains(r"(?<!항소)취하|각하|기각|원고패|패소") # contains

집행권원 = event[구분조건 & 사건번호조건 & 종국결과조건][["채무자키","새채무자키","법조치키","대상자구분","대상자명","법조치구분","법조치세부","관할법원","사건번호","사건구분","종국결과","종료일","종국결과일","확정일"]].copy()
# 날짜가 어느 행에 있을지 모르므로 중복값을 제거하지 않는다.

print(len(집행권원.index))

65986


In [ ]:
# # 출력
# 집행권원.to_excel(join(wd, basedate, "집행권원.xlsx"), index=False)

In [ ]:
# # 금감원용 지급명령 등 신청일 작성 : 강제집행, 사해행위 모두 포함하여 작성하므로 그냥 법조치 읽어서 금감원 파일에서 작성
# 집행권원신청일 = 집행권원[["새채무자키", "접수일"]].copy()

# # 접수일 역정렬, 새채무자키 중복제거
# 집행권원신청일.sort_values("접수일", ascending=False, inplace =True)
# 집행권원신청일.drop_duplicates(subset='새채무자키', keep='first', inplace=True)

# # raw_data에 열추가(merge)
# raw_data = raw_data.merge(집행권원신청일, on="새채무자키", how="left").fillna("")

In [ ]:
# 중단사유(타채)만 모으기 

# filter1 : 압류사건(타채)
사건번호조건 = event.사건구분=="타채"
압류사건 = event[사건번호조건][["채무자키","새채무자키","법조치키","대상자구분","대상자명","법조치구분","법조치세부","관할법원","사건번호","사건구분","접수일","종국결과","종료일","종국결과일","확정일","최종진행내용"]].copy()
압류사건개수 = len(압류사건)
# filter2 : 압류유지, 압류해지
# 1) 법조치구분, 법조치세부, 종국결과, 최종진행내용에 해제|해지가 포함된 것의 새채무자키와 사건번호를 추출
condition = (
    (압류사건['법조치구분'].str.contains('해제|해지', na=False)) |
    (압류사건['법조치세부'].str.contains('해제|해지', na=False)) |
    (압류사건['종국결과'].str.contains('해제|해지', na=False)) |
    (압류사건['최종진행내용'].str.contains('해제|해지', na=False))
)

압류해지 = 압류사건[condition].copy()
압류해지사건번호 = 압류해지[['새채무자키', '사건번호']]
압류해지사건번호 = 압류해지사건번호.drop_duplicates() # 기본값이 전체열 중복임
압류해지개수 = len(압류해지사건번호)
# 2) 압류사건 중 해제사건을 새채무자키와 사건번호를 기준으로 제외하기(관할법원은 없는 경우 있어서 새채무자키를 사용함)
merged = 압류사건.merge(압류해지사건번호, on=['새채무자키', '사건번호'], how='left', indicator=True) # indeicator :어느 데이터프레임에 속하는지를 '_merge'칼럼에 표시
압류유지 = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1) # 8월말 기준 both는 375개(해지가 직접 포함된 사건은 330개였음) 
압류유지개수 = len(압류유지)

# filter3 : 압류 : 종국결과 인용 (이걸 마지막에 해야 해지사건을 제대로 제외할 수 있음. 해제사건인데 종국결과가 없을 수도 있으니)
압류 = 압류유지[압류유지.종국결과.str.contains(r"결정|승소|원고승|인용|확정|발급|이행권고|지급명령|화해권고") & ~압류유지.종국결과.str.contains(r"(?<!항소)취하|각하|기각|원고패|패소")] # contains
중복제거전 = len(압류)
# 중복제거 ###################################
# # 날짜는 상관없고, 채무자별로 최신 타채 하나만 있으면 되므로 새채무자키 기준으로 중복값 제거(정렬되어있어 최신값이 남는다.)
압류 = 압류.drop_duplicates('새채무자키') 
####################################
압류개수 = len(압류)

print(f"총타채사건 : {압류사건개수}, 압류해지 : {압류해지개수}, 압류유지 : {압류유지개수}, 해지단어없지만 해지사건이라 제외된 건 : {압류사건개수 - 압류유지개수 - 압류해지개수}, 종국결과있는압류 : {압류개수}, 중복값 : {중복제거전 - 압류개수}") # 중복값때문에 숫자는 맞아 떨어지지 않음

총타채사건 : 30449, 압류해지 : 2769, 압류유지 : 27134, 해지단어없지만 해지사건이라 제외된 건 : 546, 종국결과있는압류 : 20610, 중복값 : 0


In [53]:
압류["접수일"] = pd.to_datetime(압류.접수일, errors='coerce')
특정시기압류건 = 압류[압류.사건번호.str.match('20(18|19|20|21)') | (pd.to_datetime('2018-01-01')<=압류.접수일) & (압류.접수일<=pd.to_datetime('2021-12-31'))]
특정시기압류건 = 특정시기압류건.sort_values(["새채무자키", "채무자키", "사건번호"])

In [54]:
functions.save_df_to_excel_underline(특정시기압류건, join(wd, basedate, "2018~2021_압류인용건.xlsx"), key_columns_no=2)

In [46]:
특정시기압류건.query('채무자키 =="20423305"')

,채무자키,새채무자키,법조치키,법조치구분,법조치세부,관할법원,사건번호,사건구분,접수일,종국결과,종료일,종국결과일,확정일,최종진행내용
11329,20423305,n1000074,20697197,압류,예금,서울중앙지방법원,2020타채128056,타채,2020-11-05,인용,2020-11-09,2020-11-09,0,채무자1 정순영에게 (채무자용)채권압류명령 및 추심명령결정정본 송달
9191,20423305,n1000074,20697572,압류,예금,서울중앙지방법원,2021타채120361,타채,2021-07-21,인용,2021-07-23,2021-07-23,0,채무자1 정순영에게 (채무자용)채권압류명령 및 추심명령결정정본 송달


In [42]:
압류해지.query('채무자키 =="20423305"')

,채무자키,새채무자키,법조치키,법조치구분,법조치세부,관할법원,사건번호,사건구분,접수일,종국결과,종료일,종국결과일,확정일,최종진행내용


In [51]:
event.columns[event.columns.str.contains('대상')]

Index(['대상자구분', '대상자명'], dtype='object')

In [ ]:
# 압류.to_excel(join(wd, basedate, "압류.xlsx"), index=False)

In [65]:
############## 1) 집행권원 -> 새채무자키 기준 시효완성일, 시효사유 작성
새채무자키 = []
시효완성일 = []
시효사유 = []
grouped_집행권원 = 집행권원.groupby('새채무자키')

for i, v in tqdm(grouped_집행권원, total=len(grouped_집행권원)) : 
    # DataFrame을 사건번호를 기준으로 그룹화, 그룹화하면 다시 정렬되므로 재정렬하거나 가장 마지막것으로
    grouped = v.groupby('사건번호')
    # 첫 번째 그룹의 값 확인 (가장 최근 집행권원 = 사건번호(그룹키값) 오름차순 정렬 되어있으므로 마지막 그룹)
    temp = grouped.get_group(list(grouped.groups.keys())[-1])
    
    최신확정일 = temp.확정일.max()
    최신종료일 = temp.종료일.max()
    최신종국결과일 = temp.종국결과일.max()
    최신사건년도 = str(temp.사건번호.str[:4].astype(int).max())
    # 일자가 있고, 사건접수년도와 3년 미만 차이나는 것만, 즉 유효한 일자들만 '최신'에 저장
    최신 = [x for x in [최신확정일, 최신종료일, 최신종국결과일] if (len(x) == 10) & ((int(x[:4]) - int(최신사건년도)) < 3 )]
    최신.append(최신사건년도)
    최신날짜 = sorted(최신, reverse=True)[0]

    # 시효 확정 : +10년 하되, 연도만 있는 경우 월일채우기(기준일basedate : yymmdd)
    if len(최신날짜) == 10 : #확정일, 종료일, 종국결과일 중 가장 최근 날짜
        시효 = str(int(최신날짜[:4]) + 10) + 최신날짜[4:]
    else : # 사건접수년도만 있는 경우
        시효 = str(int(최신날짜[:4]) + 10) + "-" + basedate[2:4] + "-" + basedate[4:]
    
    # 채무자별 시효 및 시효기준 작성
    새채무자키.append(i)
    시효완성일.append(시효)
    시효사유.append(temp.관할법원.values[0] + " " + temp.사건번호.values[0])

법조치시효 = pd.DataFrame({"새채무자키" : 새채무자키, "시효완성일" : 시효완성일, "시효사유":시효사유})

100%|██████████| 29113/29113 [00:17<00:00, 1654.66it/s]


In [66]:
# 압류 건 시효사유수정, 시효완성월 추가하기(법조치시효도, 압류도 새채무자키 중복값 없음)
최종시효 = 법조치시효.merge(압류[["새채무자키",'관할법원','사건번호']], on='새채무자키', how='outer', indicator=True)
최종시효["시효완성월"] = np.where(최종시효['_merge']=='left_only', '', '중단')
최종시효["시효사유"] = np.where(최종시효['_merge']=='left_only', 최종시효.시효사유, 최종시효.관할법원 + " "+ 최종시효.사건번호)
최종시효.drop(['관할법원', '사건번호', '_merge'], axis=1, inplace=True)

In [67]:
# raw_data와 시효데이터 병합
merged_raw_ori = pd.merge(raw_data, 최종시효, on='새채무자키', how='left', suffixes=['','_'])

In [68]:
# -- 코드 수정시 재시작점 --------------------------------------------
merged_raw = merged_raw_ori.copy()

# 날짜값으로 변경
merged_raw["시효완성일"] = pd.to_datetime(merged_raw["시효완성일"], format="%Y-%m-%d")  # 시효완성일을 날짜 형식으로 변환
merged_raw["시효완성일_"] = pd.to_datetime(merged_raw["시효완성일_"], format="%Y-%m-%d", errors="coerce")  # 잘못된게 없는데 에러가 났다. 에러부분은 NaT로 처리. 그런데 비교해보니 달라진 거 없었음

# not조정이면서 : 새채무상태가 개인,법인,보증인정상,시효,종료처리요망인 것
# 시효 우선 확인 목록1 : 전산시효가 시효완성월이 시효만료
# 시효 우선 확인 목록2 : 전산시효가 당월도래, 1개월 인 것 # 중단은 중단으로 되어있음
# 시효 우선 확인 목록3 : 전산시효가 확장시효보다 30일 이상차이나면서 시효사유에 '입금' 이 없는 경우
cond_base = merged_raw.새채무상태.isin(["개인","법인","보증인정상","시효","종료처리요망"])
cond1 = merged_raw.시효완성월=="시효만료"
cond2 = merged_raw.시효완성월.isin(["당월도래","1개월","2개월"])
cond3 = ((merged_raw.시효완성일 - merged_raw.시효완성일_).dt.days > 30) & (~merged_raw.시효사유.str.contains("입금")) # 30일 이상 차이
cond4 = merged_raw.시효완성일 < merged_raw.시효완성일_ # 계륵같은 조건,, 단독사용

conds = [cond_base & cond1, cond_base & cond2, cond_base & cond3, cond4]
values = ["전산시효만료", "전산시효임박", "전산시효근거없음", "시효확장"]
default_value = ""

merged_raw["시효확인필요"] = np.select(conds, values, default_value)

# 다시 문자열로
merged_raw["시효완성일"] = merged_raw["시효완성일"].apply(lambda x: pd.Timestamp(x).strftime("%Y-%m-%d") if pd.notna(x) else "")
merged_raw["시효완성일_"] = merged_raw["시효완성일_"].apply(lambda x: pd.Timestamp(x).strftime("%Y-%m-%d") if pd.notna(x) else "")

In [88]:
# 덮어쓰기 전 저장(수정전과 수정후 데이터 모두 있음) - 참고용으로 당분간 저장
merged_raw.to_excel(join(wd, basedate, "[ref]시효_수정전vs수정후.xlsx"), index=False)

In [69]:
# 시효만료나 임박건에 한해 더 나중 시효완성일로 덮어쓰기, 시효사유 수정
시효수정cond = cond_base & (cond1 | cond2) & cond4
merged_raw['시효완성일'] = np.where(시효수정cond, merged_raw.시효완성일_, merged_raw.시효완성일)
merged_raw['시효사유'] = np.where(시효수정cond, merged_raw.시효사유_, merged_raw.시효사유)

In [70]:
# 마지막 작업 : 시효완성월 전체 수정(새시효완성일에 대해)
기준일 = pd.Timestamp("20"+basedate[:4]+"01")

# 함수벡터화
merged_raw["시효완성일"] = pd.to_datetime(merged_raw["시효완성일"])  # 시효완성일을 날짜 형식으로 변환 # 위에서 했다.
merged_raw["시효완성월_계산"] = (merged_raw["시효완성일"].dt.year - 기준일.year) * 12 + (merged_raw["시효완성일"].dt.month - 기준일.month)

# 버릴 시효완성월_ 작성
x = merged_raw["시효완성월_계산"]
conds = [(x<0) | pd.isna(x), x<1, x<2, x<3, x<4, x<5, x<6, x<12, x>=12]
values = ["시효만료", "당월도래", "1개월", "2개월", "3개월", "4개월", "5개월", "1년이내", "1년이후"]
merged_raw["시효완성월_계산"] = np.select(conds, values)

# 시효완성월과 시효사유 : 압류로 인한 중단 반영
merged_raw['시효완성월'] = np.where((merged_raw.시효완성월_=="중단") | (merged_raw.시효완성월=="중단") , "중단", merged_raw.시효완성월_계산)
merged_raw['시효사유'] = np.where(merged_raw.시효완성월_=="중단", merged_raw.시효사유_, merged_raw.시효사유)

# raw_data 저장 : 타입원복 및 불필요한 열 제거
raw_data = merged_raw.iloc[:,:-5] # 시효확인필요, 시효완성일_, 시효사유_, 시효완성월_, 시효완성월_계산 제거
raw_data["시효완성일"] = raw_data["시효완성일"].apply(lambda x: pd.Timestamp(x).strftime("%Y-%m-%d") if pd.notna(x) else "")

#### 칼럼 순서 및 저장

In [71]:
# 컬럼 순서 
raw_data.insert(8, "새보증인상태", raw_data.pop('새보증인상태'))
raw_data.insert(9, "반영OPB", raw_data.pop('반영OPB'))

In [72]:
# # 요약 

# 특정 POOL
# pool계좌키 = pd.read_excel("c://users/sl/Desktop/계좌키.xlsx", dtype={"계좌키":str})
# pool = pd.merge(raw_data,pool계좌키, on='계좌키', how='inner')
# result = pool.groupby('새채무상태').agg({'계좌키':'count', '반영OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])
# result
# 전체
# reindex에 있는 것만 집계되네 
전체요약 = raw_data.groupby('새채무상태').agg({'계좌키':'count', '반영OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)', \
    '개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','보증인정상','상각','시효','가지급계좌','공채권','종료처리요망','종료']).fillna(0) # .applymap('{:,.0f}'.format)
display(전체요약)

# 전체 저장하기
# 파일명 : 새채무자키에 따라 보증인상태 추렸는지
if 새채무자키사용 : path_raw_data = join(wd, basedate, 원데이터파일명["전체"]+"_by새채무자키")
else : path_raw_data = join(wd, basedate, 원데이터파일명["전체"])

# 엑셀 파일로 저장
if not os.path.exists(path_raw_data + ".xlsx"):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(path_raw_data + ".xlsx", engine='openpyxl', **param) as writer :
    raw_data.to_excel(writer, sheet_name="리스트", index=False)
    전체요약.to_excel(writer, sheet_name='요약', index=True)

# pkl
if 새채무자키사용 : raw_data.to_pickle(join(wd, basedate, "pkl", 원데이터파일명["전체"] + "_by새채무자키" + ".pkl"))
else : raw_data.to_pickle(join(wd, basedate, "pkl", 원데이터파일명["전체"] + ".pkl"))

# 특정 매각사만 저장하기
# cond = "스마트저축-01" ###################################
# raw_data[raw_data.매각사구분==cond].to_excel(join(wd, cond+"_원데이터_"+basedate+".xlsx"), index=False)

,계좌키,반영OPB
새채무상태,,
개인,"10,519","108,920,592,867"
법인,"2,340","190,706,102,578"
개인회생(확정),"2,570","3,992,488,270"
개인회생(진행중),200,"1,637,179,335"
신용회복(개인),"7,646","15,666,243,016"
신용회복(프리),"4,281","26,147,435,850"
신용회복(진행중),134,"889,507,143"
보증인정상,542,"10,950,167,687"
상각,27,"2,793,805,401"


### 개회, 신복 원데이터 만들기

In [74]:
# 개인회생 : 제출용 시트 작성하기
re_d = rehabilitation_d.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가미인가","현재결과", \
    "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일",\
    "총분납금","총분납입금","총분납잔금","분납미납금","분납잔금","총분납회차","현재회차","납입회차","잔여회차","미납회차","현재원금","현재법비용",
    "분납메모", "분납비고", "분납기타", "기타금액"]].copy() # ,"초회여부", "일시변제금액", "재산처분금액"
re_grt = rehabilitation_grt.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가미인가","현재결과", \
    "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일",\
    "총분납금","총분납입금","총분납잔금","분납미납금","분납잔금","총분납회차","현재회차","납입회차","잔여회차","미납회차","현재원금","현재법비용",\
    "분납메모", "분납비고", "분납기타", "기타금액"]].copy() #,"초회여부", "일시변제금액", "재산처분금액"


###### 260613 코드 종결. 초과입금이 미래회차가 선입금 된 것이 아니라 과거 미납회차가 들어온 것으로 보고 전산 그대로 간다. 초과입금을 빼주려거든 기준데이터에서 아예 새 칼럼으로 수정하자. 반영OPB도 마찬가지고
# 실제 총분납잔금 = 전산 총분납잔금 - 전산 초과입금 (초과입금은 현재 회차 이후 입금액을 따로 구분한 것이고, 그것을 전산에서는 더해서 분납잔금을 계산했다. 즉 회차기준으로 작성한 것)
# 총분납입금은 회차 계산없이 실제 입금한 금액
# re_d["총분납잔금"] = re_d["총분납금"] - re_d["총분납입금"] ############### fillna때문에 오류날수도 , 기준데이터 말고 여기서 하는 게 맞다.
# re_grt["총분납잔금"] = re_grt["총분납금"] - re_grt["총분납입금"]
# 이 코드 살린다면 반영opb를 총분납잔금으로 재설정해줘야. 통합건, 연납건은 어떻게 하든 코드적인 문제 없음 확인ㅇ


# 계좌 분리 후 비고란을 작성했던 때의 코드 ---------------
# if "비고" in rehabilitation.columns :
#     re_d = rehabilitation_d.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","분납메모"]]
#     re_grt = rehabilitation_grt.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","분납메모"]]
# else : # 계좌키분리 안 하면 '비고'란 없음
#     re_d = rehabilitation_d.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용"]]
#     re_grt = rehabilitation_grt.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용"]]
# 계좌 분리 후 비고란을 작성했던 때의 코드 ---------------
# re_d.insert(0, "순번", range(1,len(re_d)+1))
# re_grt.insert(0, "순번", range(1,len(re_grt)+1))
# 저장
with pd.ExcelWriter(join(wd, basedate, "2."+ company +"_개인회생_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    re_d.to_excel(writer, sheet_name="차주", index=False)
    re_grt.to_excel(writer, sheet_name="보증인", index=False)

# 개인회생 pkl : 개인회생 기준데이터
re_d.to_pickle(join(wd, basedate, "pkl", 원데이터파일명["개회차주"] + ".pkl"))
re_grt.to_pickle(join(wd, basedate, "pkl", 원데이터파일명["개회보증인"] + ".pkl"))

In [78]:
# 신용회복 : 제출용 시트 작성하기
# 기준데이터에서 삭제한 것이 있을 때 유의미 & 차주 보증인 구분

# # "NaT" 처리 - 제대로 됐으면 안 해도 되는건데.. 뭔가 꼬였을 때
# credit_d.replace({"NaT": ""}, inplace=True)
# credit_grt.replace({"NaT": ""}, inplace=True)

# # 날짜열 문자열로 : 업로드결과를 사용할 때는 필요하지 않음
# date_cols = credit_d.select_dtypes(include='datetime').columns
# credit_d[date_cols] = credit_d[date_cols].astype(str)
# date_cols = credit_grt.select_dtypes(include='datetime').columns
# credit_grt[date_cols] = credit_grt[date_cols].astype(str)

# # 저장
with pd.ExcelWriter(join(wd, basedate, "2."+ company +"_신용회복_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    credit_d.to_excel(writer, sheet_name="차주", index=False)
    credit_grt.to_excel(writer, sheet_name="보증인", index=False)
# 신용회복 pkl : 신용회복 기준데이터
credit_d.to_pickle(join(wd, basedate, "pkl", 원데이터파일명["신복차주"] + ".pkl"))
credit_grt.to_pickle(join(wd, basedate, "pkl", 원데이터파일명["신복보증인"] + ".pkl"))

### POOL별 엑셀 파일 만들기

In [125]:
# wd = r"D:\3.자산\전산 dataset\230430\매각환매 채무상태 수정후"
# basedate = "230430"
# dir = wd  
dir = join(wd, basedate)

#### 바로 작업시에는 이 셀 pass---------------------------------

In [126]:
# 1.개인회생
re_d = pd.read_pickle(join(dir, "pkl", 원데이터파일명["개회차주"])+".pkl")
re_grt = pd.read_pickle(join(dir, "pkl", 원데이터파일명["개회보증인"])+".pkl")

# 2.신용회복
credit_d = pd.read_pickle(join(dir, "pkl", 원데이터파일명["신복차주"])+".pkl")
credit_grt = pd.read_pickle(join(dir, "pkl", 원데이터파일명["신복보증인"])+".pkl")

# 3.전체(무담보)
###############################
새채무자키사용 = True
###############################
if 새채무자키사용 : 
    raw_data = pd.read_pickle(join(dir, "pkl", 원데이터파일명["전체"]+"_by새채무자키.pkl"))
else : 
    raw_data = pd.read_pickle(join(dir, "pkl", 원데이터파일명["전체"]+".pkl"))
#raw_data["원인서류유무"] = ""


# 3.전체 임시 - 새채무자키 사용

# files = ["2.솔림_개인회생_원데이터_"+basedate, "2.솔림_신용회복_원데이터_"+basedate, "2.솔림_전체_원데이터_"+basedate]

# # 1.개인회생
# re_d = pd.read_excel(join(dir, files[0]+".xlsx"), dtype=rehabilitation_dtype, sheet_name="차주").fillna("")
# re_grt = pd.read_excel(join(dir, files[0]+".xlsx"), dtype=rehabilitation_dtype, sheet_name="보증인").fillna("")

# # 2.신용회복
# credit_d = pd.read_excel(join(dir, files[1]+".xlsx"), dtype=credit_dtype, sheet_name="차주").fillna("")
# credit_grt = pd.read_excel(join(dir, files[1]+".xlsx"), dtype=credit_dtype, sheet_name="보증인").fillna("")

# # 3.무담보
# raw_data = pd.read_excel(join(dir, files[2]+".xlsx"), dtype=account_dtype, sheet_name="리스트").fillna("")

# 날짜타입 칼럼
# date_cols = credit_d.select_dtypes(include='datetime').columns
# credit_d[date_cols] = credit_d[date_cols].astype(str)
# date_cols = credit_grt.select_dtypes(include='datetime').columns
# credit_grt[date_cols] = credit_grt[date_cols].astype(str)

# credit_d.replace({pd.NaT: ""}, inplace=True)
# credit_grt.replace({pd.NaT: ""}, inplace=True)

#### 풀 정의

In [ ]:
###################################################
# 무담보 채무상태 추가처리 : 파산(기각제외), 사망 종료로 제외하기 
# 일시적으로 적용할 작업있으면 여기서 처리하자
# raw_data.loc[(raw_data.채무상태.str.contains(r"파산(?!\(기각\))|사망")) | (raw_data.담당자.str.contains(r"파산(?!\(기각\))|사망")) ,"새채무상태"] = "종료"
###################################################

In [ ]:
# ##### p2 임시 ########

# ###################################################################################
# # P1 수정전(7558건 모두 P1) OR 대성
# path_pooldata = join(r"D:\3.자산\POOL별 관리자산\솔림\POOL 14 번자료 (우담-01, 우리종합금융)\0823_대출연장\대출연장서류_웰컴","9.POOL14_데이터_230731_회계법인 발송.xlsx" ) 
# # # P1 수정후(1월 종료건은 무차입으로)
# # path_pooldata = join(r"D:\3.자산\POOL별 관리자산", company, "POOL정보-P1수정.xlsx" ) 
# ###################################################################################
# pooldata = pd.read_excel(path_pooldata, sheet_name="무담보", dtype={"채무자키":str,"계좌키":str})

# pool_dict = {"POOL8_무담보" : pooldata["계좌키"]}

In [127]:
# 풀데이터 읽기
###################################################################################
path_pooldata = join(r"D:\3.자산\POOL별 관리자산", company, "POOL정보.xlsx" ) 
###################################################################################
pooldata = pd.read_excel(path_pooldata, sheet_name="POOL정리", dtype={"계좌키":str})

# 전체계좌건수 비교
print(f'계좌수:{len(account)}, 풀데이터계좌수{len(pooldata)}')
if len(account) == len(pooldata) : 
    print("풀데이터에 예수금 제외한 모든 계좌가 반영되어 있음")
else : 
    print(f"풀데이터와 계좌데이터의 수가 맞지 않음. 풀데이터:{len(pooldata)}건, 계좌:{len(account)}건. 차이{len(pooldata)-len(account)}")

계좌수:78406, 풀데이터계좌수78406
풀데이터에 예수금 제외한 모든 계좌가 반영되어 있음


In [133]:
# POOL별 채무자키 # 1 이상은 True
# 대성 : p2, 무차입
if company == "솔림" : # 솔림
    pool_dict = {
                "POOL1" : pooldata.query('P1==1')["계좌키"],
                # "POOL2" : pooldata.query('P2==1')["계좌키"], # 완전매각
                "POOL3" : pooldata.query('P3==1')["계좌키"],
                # "POOL4" : pooldata.query('P4==1')["계좌키"],
                "POOL5" : pooldata.query('P5==1')["계좌키"],
                "POOL6" : pooldata.query('P6==1')["계좌키"],
                # "POOL6(단독)" : pooldata.query('P6==1 and 소속풀수==1')["계좌키"], ###
                # "POOL7" : pooldata.query('P7==1')["계좌키"], # 완전매각
                "POOL8" : pooldata.query('P8==1')["계좌키"],
                # "POOL9" : pooldata.query('P9==1')["계좌키"], # 완전매각
                "POOL10" : pooldata.query('P10==1')["계좌키"],
                "POOL11" : pooldata.query('P11==1')["계좌키"],
                # "POOL12" : pooldata.query('P12==1')["계좌키"], # 완전상환
                "POOL13" : pooldata.query('P13==1')["계좌키"],
                "POOL15" : pooldata.query('P15==1')["계좌키"],
                # "무차입" : pooldata.query('소속풀수==0')["계좌키"],
                # "전체" : pooldata["계좌키"],
                # "발췌" : pooldata.query('발췌==1')["계좌키"],
                # "POOL4" : pooldata.query('P4_매각예정==1')["계좌키"],
                
                #-----------------------------------------------------------------------
                
                # # 다른 거 모두 작성 후 마지막에 할 거(raw_data 수정되므로)
                # "POOL14" : pooldata.query('P14==1')["계좌키"], ###### 별도작업
                # # "10,14,15,발췌 통합" : pooldata.query('발췌==1 or P10==1 or P14==1 or P15==1')["계좌키"], ###### 별도작업
                # # "POOL10" : pooldata.query('P10매각포함==1')["계좌키"], ###### 별도작업
                }
else : # 대성
    pool_dict = {
                "무차입" : pooldata.query('소속풀수==0')["계좌키"],
                "POOL2" : pooldata.query('P2==1')["계좌키"],
                "전체" : pooldata["계좌키"],
                
                # "매각예정" : pooldata.query('매각예정==1')["계좌키"],
                }
# p10 2023년 06월 아래 계좌는 제외되어 대출연장 확정됨에 따라, 별도로 처리할 필요 없음
# if "POOL10" in pool_dict.keys() : 
#     # P10(매각포함-기본값) 작업시
#     # 해당리스트는 P4및 무차입으로 정리. P10에도 포함시켜야 하니 P10만 나중에 따로 하고 그때 이 셀 실행하면 됨
#     대성매각 = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\예외\P10_대성매각건.xlsx", dtype={'계좌키':str})
#     raw_data["채권상태"] = np.where(raw_data.계좌키.isin(대성매각.계좌키), "종료", raw_data.채권상태)
#     raw_data["새채무상태"] = np.where(raw_data.계좌키.isin(대성매각.계좌키), "종료", raw_data.새채무상태)
#     raw_data["채무상태"] = np.where(raw_data.계좌키.isin(대성매각.계좌키), "종료", raw_data.채무상태)
#     raw_data["담당자"] = np.where(raw_data.계좌키.isin(대성매각.계좌키), "종결(매각)", raw_data.담당자)
    
if ("POOL14" in pool_dict.keys()) or ("10,14,15,발췌 통합" in pool_dict.keys()) : 
    개인법인수정목록 = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\예외\P14_개인법인수정목록.xlsx", dtype=grt_dtype)
    개인법인수정목록 = 개인법인수정목록.drop(columns=["채무자키","보증인키"]) # 일단 안 쓰니 드랍한다. 보증인새창에서 연령같은 거 읽어오는 식으로 코드 바꿀때나 필요
    # 해가 바뀌면 연령만 엑셀파일 수정
    
    # 개인법인수정목록에 새채무상태,채권상태 넣기 : 법인이면 개인으로 바꿔주고, 다른상태(종결이나 종결요청만 가능)로 바뀌면 그대로 적어준다.
    temp새채무상태, temp채권상태 = [], []
    for i, v in 개인법인수정목록.iterrows() : 
        원래새채무상태 = raw_data.query('계좌키==@v.계좌키').새채무상태.values[0]
        원래채권상태 = raw_data.query('계좌키==@v.계좌키').채권상태.values[0]
        if 원래새채무상태 == "법인" : 
            temp새채무상태.append("개인")
            temp채권상태.append("정상")
        else : 
            temp새채무상태.append(원래새채무상태)
            temp채권상태.append(원래채권상태)
    개인법인수정목록["새채무상태"] = temp새채무상태
    개인법인수정목록["채권상태"] = temp채권상태
    
    # 개인법인수정목록의 계좌키를 기준으로 raw_data와 병합 (접미사 추가)
    raw_data = pd.merge(raw_data, 개인법인수정목록, on='계좌키', how='left', suffixes=('', '_수정'))

    # 수정목록에 해당하는 열이 있으면 raw_data의 열을 수정목록으로 대체
    raw_data['성명'] = raw_data['성명_수정'].combine_first(raw_data['성명'])
    raw_data['주민등록번호'] = raw_data['주민등록번호_수정'].combine_first(raw_data['주민등록번호'])
    raw_data['개인법인'] = raw_data['개인법인_수정'].combine_first(raw_data['개인법인'])
    raw_data['연령'] = raw_data['연령_수정'].combine_first(raw_data['연령'])
    raw_data["새채무상태"] = raw_data['새채무상태_수정'].combine_first(raw_data['새채무상태'])

    # 개인법인수정목록 열 삭제
    raw_data = raw_data.drop(columns=['성명_수정', '주민등록번호_수정', '개인법인_수정', '연령_수정', '새채무상태_수정'])

##### 풀별 내부/발송 파일 만들기
- 일부 풀만 작업할 경우 위 pool_dict 주석처리를 통해

In [129]:
def summary_comment (ws1, file_kind, comp_closing, basedate) : 
    file_kind = file_kind
    start_row = None
    data = ["■ 자산확정일 : 20" + basedate[0:2]+"-"+basedate[2:4]+"-"+basedate[4:],
        "",
        "■ OPB 작성기준(*기준일까지의 입금내역 모두 반영됨)",
        "무담보 : 미상환원금잔액(현재원금+현재법비용)",
        "개인회생(인가) : 총분납잔금",
        "개인회생(미인가) : 미상환원금잔액(현재원금+현재법비용)",
        "***신용회복 : 기준일의 정확한 데이터(진행상태,회차,금액) 반영을 위해 기준일이 금~일요일이 아닌 경우, 약식데이터와 정식데이터를 병합하여 작성",
        "신용회복(개인) : (원금균등) 상환후잔액 → 원금균등채무액 → 조정후원금 → 조정전원금",
        "신용회복(프리) : (원리금균등) 상환후잔액 → 원금균등채무액 → 조정후원금 → 조정전원금",
        "신용회복(미확정) : 조정전원금 + 조정전비용", # 9번 요소 
        "",
        "■ 제외(종료)기준 1 - 확실한 종료건",
        "  ▶ 종결일이 있거나 계좌메모에 종결여부가 표시되어있을 때",
        "  ▶ 채무상태나 담당자 값이 다음을 포함할 때 : " + comp_closing["종료"],
        "",
        "■ 제외(종료)기준 2 - 비정상건(매각 불가)",
        "  ▶ [보증인정상] : 차주는 [종결처리요망]에 해당하나",
        "     정상 상태의 보증인이 있을때",
        "  ▶ [시효] : 채무상태나 담당자 값이 다음을 포함할 때 : " + comp_closing["시효"],
        "  ▶ [공채권] : 계좌 통합신고 등의 사유로 OPB가 다른 계좌에 반영되어버린 경우",
        "  ▶ [가지급계좌] : 주채무는 죽고 보증채무는 살아있으나, 가지급계좌라서 보증인등록이 되어있지 않은 건(23년11월까지는 종결처리요망으로 분류됨)",
        "  ▶ [종료처리요망] : 채무상태나 담당자 값이 다음을 포함하거나 : " + comp_closing["면책"] + "|" +comp_closing["불가"],
        "     차주가 법인이면서 정상 상태의 보증인이 한 명도 없을 때",
        "",
        "■ 시효 추가작업 - 지급명령, 본안소송, 타채",
        "  ▶ [기본기준] : 새채무자키, 법조치 대상자가 보증인이라도 차주로 간주",
        "  ▶ [시효연장] : ",
        "     지급명령, 본안소송 중 가장 나중 사건의 사건연도 3년 이내 [종료일,종국결과일,확정일] 중 가장 최근일 + 10년",
        "     종료일, 종국결과일, 확정일 데이터가 없는 경우 사건연도와 자산정리 기준월일 + 10년",
        "     종국결과 - '결정|발급|승소|원고승|일부승|이행권고|화해|인용|지급명령|확정|항소기각|항소취하간주|조정(?!불)'",
        "     수정대상 : 시효완성월이 시효만료~2개월 인 채권 중 위 계산에 따라 시효가 연장된 경우",
        "  ▶ [시효중단] : ",
        "     타채 사건들 중 해지 또는 해제 키워드가 [법조치구분,법조치세부,종국결과,최종진행내용]에 있는 경우 해당 사건은 완전 제외",
        "     결정|승소|원고승|인용|확정|발급|이행권고|지급명령|화해권고",
        "■ 칼럼 설명",
        "  ▶ [분납연체일] : 당사 약정분납 체결자 중 분납진행중이고, 분납연체일 90일 이하인 채권의 분납연체일",
        "■ 반영된 입금내역",
        "  ▶ 컷오프일까지의 입금내역이 모두 반영되어 있음"
        ]
    if file_kind == "발송" :
        start_row = 16
        data = data[:10] 
    else : 
        start_row = 18
    for i, row_data in enumerate(data) :
        ws1.cell(row=start_row + i, column=1,value=row_data)
    return ws1

def set_col_width_ws1(ws1) : 
    ws1.column_dimensions["A"].width = 18
    ws1.column_dimensions["B"].width = 10
    ws1.column_dimensions["C"].width = 14
    ws1.column_dimensions["D"].width = 10
    ws1.column_dimensions["E"].width = 14

In [134]:
pd.set_option('mode.chained_assignment', None)
#####################################
output_dir = join(dir,"output")
#####################################
if not os.path.exists(output_dir) : os.mkdir(output_dir)
raw_data.drop(["은행대출원금"], axis=1)  # 은행대출원금 삭제

for pool_kind, pool_keys in tqdm(pool_dict.items(), total=len(pool_dict))  : 
    
    ###### 내부용 만들기############################################################
    file_kind = "내부"
    # 풀 원데이터
    pool_raw = raw_data[raw_data.계좌키.isin(pool_keys)].copy()
    # 풀 무담보, 종료건
    pool_무담보 = pool_raw[pool_raw["새채무상태"].isin(["개인","법인"])]
    pool_비정상 = pool_raw[pool_raw["채권상태"]=="비정상"]
    pool_종료 = pool_raw[pool_raw["채권상태"]=="종료"]
    
    # 풀 조정건 : 새채무상태상 개회, 신복인 것만 고르기(기준데이터에 종결건 포함돼도 ok)
    re_d_keys = pool_raw[pool_raw["새채무상태"].str.contains("회생")].계좌키
    credit_d_keys = pool_raw[pool_raw["새채무상태"].str.contains("신용")].계좌키
    grt_keys = pool_raw[pool_raw["채권상태"] != "종료"].계좌키 # 종료채권이 아닌 것 : 조정중인 보증채무 나열하기 위함
        
    pool_re_d = re_d[re_d.계좌키.isin(re_d_keys)]
    pool_re_grt = re_grt[re_grt.계좌키.isin(grt_keys)] 
    pool_credit_d = credit_d[credit_d.계좌키.isin(credit_d_keys)]
    pool_credit_grt = credit_grt[credit_grt.계좌키.isin(grt_keys)]

    #순번 넣기
    pool_종료.insert(0, "순번", range(1,len(pool_종료)+1))
    pool_비정상.insert(0, "순번", range(1,len(pool_비정상)+1))
    pool_무담보.insert(0, "순번", range(1,len(pool_무담보)+1))
    pool_re_d.insert(0, "순번", range(1,len(pool_re_d)+1))
    pool_re_grt.insert(0, "순번", range(1,len(pool_re_grt)+1))
    pool_credit_d["순번"] = range(1,len(pool_credit_d)+1)
    pool_credit_grt["순번"] = range(1,len(pool_credit_grt)+1)

    ####### 요약시트[시작] - 새채무상태 종류 달라진 경우 수정필요 ##################################################
    summary = pool_raw.groupby('새채무상태').agg({'계좌키':'count', '반영OPB':'sum'}).reindex( \
        index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','보증인정상','상각','시효','가지급계좌','공채권','종료처리요망','종료'])
    ############################################################################################################
    summary.rename(columns={'계좌키':'건수', '반영OPB':"OPB"}, inplace=True)

    # 보증인 합산 : summary의 index가 수정되면 이것도 수정해야
    summary["건수(보증인)"] = [
        0,0, # 개인, 법인은 그대로
        pool_re_grt[pool_re_grt['인가미인가']=="인가"].계좌키.count(), # 보증인은 없는 경우 있어서 그룹화하면 index에러날 수 있음
        pool_re_grt[pool_re_grt['인가미인가']=="미인가"].계좌키.count(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환구분=="개인")].계좌키.count(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환구분=="프리")].계좌키.count(),
        pool_credit_grt[pool_credit_grt.진행구분=="미확정"].계좌키.count(), 
        0, 0, 0, 0, 0, 0, 0 # 비정상6건과 종료
    ]
    summary["OPB(보증인)"] = [
        0,0, # 개인, 법인은 그대로
        pool_re_grt[pool_re_grt['인가미인가']=="인가"].반영OPB.sum(),
        pool_re_grt[pool_re_grt['인가미인가']=="미인가"].반영OPB.sum(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환구분=="개인")].반영OPB.sum(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환구분=="프리")].반영OPB.sum(),
        pool_credit_grt[pool_credit_grt.진행구분=="미확정"].반영OPB.sum(),
        0, 0, 0, 0, 0, 0, 0 # 비정상6건과 종료
    ]

    # "종결" 행 제외한 열의 합계 계산
    row_sum = summary[summary.index != "종료"].sum()
    summary.loc["합계(종료제외)"] = row_sum ###############################################################
    summary = summary.reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)',\
        '보증인정상','상각','시효','가지급계좌','공채권','종료처리요망',"합계(종료제외)",'종료'])
    ######################################################################################################

    summary = summary.fillna(0).astype(dtype='int64')
    summary.rename_axis("구분", inplace=True)
    

    ######### 엑셀 쓰기
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "요약"
    ws2 = wb.create_sheet("무담보")
    ws3 = wb.create_sheet("개인회생_차주")
    ws4 = wb.create_sheet("개인회생_보증인")
    ws5 = wb.create_sheet("신용회복_차주")
    ws6 = wb.create_sheet("신용회복_보증인")
    ws7 = wb.create_sheet("비정상")
    ws8 = wb.create_sheet("종료")

    # 시트에 데이터 쓰기
    for r in dataframe_to_rows(summary, index=True, header=True): ws1.append(r)
    # a1에 구분 넣고 요약셀 2행 삭제
    ws1['a1'] = "구분"
    ws1.delete_rows(2)

    # 코멘트 작성하기
    ws1 = summary_comment(ws1, file_kind, comp_closing, basedate)

    for r in dataframe_to_rows(pool_무담보, index=False, header=True): ws2.append(r)
    for r in dataframe_to_rows(pool_re_d, index=False, header=True): ws3.append(r)
    for r in dataframe_to_rows(pool_re_grt, index=False, header=True): ws4.append(r)
    for r in dataframe_to_rows(pool_credit_d, index=False, header=True): ws5.append(r)
    for r in dataframe_to_rows(pool_credit_grt, index=False, header=True): ws6.append(r)
    for r in dataframe_to_rows(pool_비정상, index=False, header=True): ws7.append(r)
    for r in dataframe_to_rows(pool_종료, index=False, header=True): ws8.append(r)

    # 스타일 지정
    # 폰트
    #font = Font(name='NN30', size=10, color='FF000000')
    # 폰트 (데이터 부분)
    font_data = Font(name='NN30', size=10)
    # 배경색
    fill_col = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6') # 칼럼명
    fill_new = PatternFill(fill_type='solid', start_color='FF49C620', end_color='FF49C620') # 칼럼명에 '새'가 들어간 것
    # 테두리 : 직선
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    for ws in [ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8] :

        # 컬럼명 스타일 적용
        for cell in ws[1]:
            cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
            if cell.value is not None and (cell.value.startswith('새') or re.search("채권상태|by|시효실사결과|원인서류유무", cell.value)) : cell.fill = fill_new
            else : cell.fill = fill_col
            cell.border = border
            cell.alignment = alignment

        # 데이터 부분 스타일 적용
        # 요약시트
        ####################
        max_row = 16 if ws.title == "요약" else ws.max_row
        ####################
        # 다른시트
        for row in ws.iter_rows(min_row=2, max_row=max_row):
            for cell in row:
                # 숫자 데이터인 경우
                if isinstance(cell.value, (int, float)):
                    cell.font = font_data
                    cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
                    cell.alignment = Alignment(horizontal='right', vertical='center')
                # 날짜 데이터인 경우
                elif isinstance(cell.value, pd.Timestamp):
                    cell.font = font_data
                    cell.number_format = 'yyyy-mm-dd'
                    cell.alignment = alignment
                # 나머지 문자열 데이터인 경우
                else:
                    cell.font = font_data
                    cell.alignment = alignment
                cell.border = border

    # 요약시트 "합계(종료제외)" 배경색
    ####################
    for cell in ws1[15]:
    ####################
        cell.fill = PatternFill(fill_type='solid', start_color='B8CCE4', end_color='B8CCE4')

    # 요약시트 나머지행 글자크기
    for row in ws1.iter_rows(min_row=10) :
        for cell in row : 
            cell.font = Font(name='NN30', size=10)
    
    # 요약시트 컬럼 너비 지정하기(단위는 char임)
    set_col_width_ws1(ws1)

    # 파일 저장 --------------------------------------------------------------------------------
    if 새채무자키사용 : wb.save(join(output_dir, pool_kind+"_데이터_"+basedate+"_"+file_kind+"_by새채무자키.xlsx"))
    else : wb.save(join(output_dir, pool_kind+"_데이터_"+basedate+"_"+file_kind+".xlsx"))


    ###### 발송용 만들기--------------------------------------------------------------------------------
    file_kind = "발송" ####

    ################# 칼럼 변경시 ##################
    무담보삭제할열후보 = ['채권상태','새채무상태','채무상태','담당자','새보증인상태','회수합계','종결일', '매입OPB',
                   '메모','시효실사결과','원인서류유무','새보증인상태by새채무자키','최종거래일','분납연체일']
    개인회생삭제할열후보 = ['분납비고','분납기타','기타금액','초회여부','일시변제금액','재산처분금액']
    무담보삭제할열 = [col for col in 무담보삭제할열후보 if col in pool_무담보.columns]
    개인회생삭제할열 = [col for col in 개인회생삭제할열후보 if col in pool_re_d.columns]
    ###############################################
    pool_무담보.drop(무담보삭제할열, axis=1, inplace=True) # '시효완성월','시효사유',
    pool_re_d.drop(개인회생삭제할열, axis=1, inplace=True)
    
    pool_무담보["성명"] = pool_무담보["성명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_무담보["주민등록번호"] = pool_무담보["주민등록번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_re_d["분납자성명인"] = pool_re_d["분납자성명인"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_re_grt["분납자성명인"] = pool_re_grt["분납자성명인"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_d["신청인명"] = pool_credit_d["신청인명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_d["신청인주민번호"] = pool_credit_d["신청인주민번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_credit_grt["신청인명"] = pool_credit_grt["신청인명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_grt["신청인주민번호"] = pool_credit_grt["신청인주민번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_종료 = pd.concat([pool_비정상, pool_종료],)
    pool_종료["순번"] = range(1,len(pool_종료)+1)
    pool_종료 = pool_종료[["순번","채무자키","계좌키","채무상태","담당자","성명","주민등록번호"]] # 담당자열 추가 040723~
    pool_종료["성명"] = pool_종료["성명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_종료["주민등록번호"] = pool_종료["주민등록번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))

    wb = Workbook()
    ws1 = wb.active
    ws1.title = "요약"
    ws2 = wb.create_sheet("무담보")
    ws3 = wb.create_sheet("개인회생_차주")
    ws4 = wb.create_sheet("개인회생_보증인")
    ws5 = wb.create_sheet("신용회복_차주")
    ws6 = wb.create_sheet("신용회복_보증인")
    ws7 = wb.create_sheet("종료")

    # 시트에 데이터 쓰기

    # 요약 시트 재작성  - 새채무상태 종류 달라지면 재작성 #############################################
    summary.loc["종료"] += summary.loc["보증인정상"] + summary.loc["시효"] + summary.loc["종료처리요망"] + summary.loc["가지급계좌"] + summary.loc["공채권"] + summary.loc["상각"]
    summary.loc["합계(종료제외)"] -= summary.loc["보증인정상"] + summary.loc["시효"] + summary.loc["종료처리요망"] + summary.loc["가지급계좌"] + summary.loc["공채권"] + summary.loc["상각"]
    summary = summary.drop(["보증인정상", "시효", "종료처리요망", "가지급계좌", "공채권", '상각'])
    summary.loc["종료","OPB"] = 0
    ###############################################################################################

    for r in dataframe_to_rows(summary, index=True, header=True): ws1.append(r)
    # a1에 구분 넣고 요약셀 2행 삭제
    ws1['a1'] = "구분"
    ws1.delete_rows(2)
    
    # 코멘트 작성하기
    ws1 = summary_comment(ws1, file_kind, comp_closing, basedate)

    for r in dataframe_to_rows(pool_무담보, index=False, header=True): ws2.append(r)
    for r in dataframe_to_rows(pool_re_d, index=False, header=True): ws3.append(r)
    for r in dataframe_to_rows(pool_re_grt, index=False, header=True): ws4.append(r)
    for r in dataframe_to_rows(pool_credit_d, index=False, header=True): ws5.append(r)
    for r in dataframe_to_rows(pool_credit_grt, index=False, header=True): ws6.append(r)
    for r in dataframe_to_rows(pool_종료, index=False, header=True): ws7.append(r)

    # 스타일 지정
    # 폰트
    #font = Font(name='NN30', size=10, color='FF000000')
    # 폰트 (데이터 부분)
    font_data = Font(name='NN30', size=10)
    # 배경색
    fill = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6')
    # 테두리
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    for ws in [ws1,ws2,ws3,ws4,ws5,ws6,ws7] :

        # 컬럼명 스타일 적용
        for cell in ws[1]: # 시트의 1행의 셀 반복
            cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
            cell.fill = fill
            cell.border = border
            cell.alignment = alignment

        # 데이터 부분 스타일 적용
        # 요약시트 ######################################
        max_row = 10 if ws.title == "요약" else ws.max_row
        #############
        # 다른시트
        for row in ws.iter_rows(min_row=2, max_row=max_row):
            for cell in row:
                # 숫자 데이터인 경우
                if isinstance(cell.value, (int, float)):
                    cell.font = font_data
                    cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
                    cell.alignment = Alignment(horizontal='right', vertical='center')
                # 날짜 데이터인 경우
                elif isinstance(cell.value, pd.Timestamp):
                    cell.font = font_data
                    cell.number_format = 'yyyy-mm-dd'
                    cell.alignment = alignment
                # 나머지 문자열 데이터인 경우
                else:
                    cell.font = font_data
                    cell.alignment = alignment
                cell.border = border
        
    # 요약시트 "합계(종료제외)" 배경색
    ####################
    for cell in ws1[9]:
    ####################
        cell.fill = PatternFill(fill_type='solid', start_color='B8CCE4', end_color='B8CCE4')
    
    # 요약시트 나머지행 글자크기
    for row in ws1.iter_rows(min_row=10) :
        for cell in row : 
            cell.font = Font(name='NN30', size=10)

    # 요약시트 컬럼 너비 지정하기(단위는 char임)
    set_col_width_ws1(ws1)
            
    # 파일 저장 ################################################################
    if 새채무자키사용 : wb.save(join(output_dir, pool_kind+"_데이터_"+basedate+"_"+file_kind+"_by새채무자키.xlsx"))
    else : wb.save(join(output_dir, pool_kind+"_데이터_"+basedate+"_"+file_kind+".xlsx"))

100%|██████████| 9/9 [01:47<00:00, 11.91s/it]


In [ ]:
############## 끝 ###################

# 한투
- 로우데이터만 읽으면 됨(풀별 작성처럼)

In [ ]:
# p11 계좌키
계좌키11 = pooldata.query('P11==1')["계좌키"]

# raw_data
원데이터11 = raw_data[raw_data.계좌키.isin(계좌키11)].copy()

# 무담보
무담보11 = 원데이터11[원데이터11["새채무상태"].isin(["개인","법인"])]

# 풀 조정건 : 기준데이터에서 불러오기
re_d_keys = 원데이터11[원데이터11["새채무상태"].str.contains("회생")].계좌키
credit_d_keys = 원데이터11[원데이터11["새채무상태"].str.contains("신용")].계좌키
    
개회11 = re_d[re_d.계좌키.isin(re_d_keys)]
신복11 = credit_d[credit_d.계좌키.isin(credit_d_keys)]

# reset_index
무담보11.reset_index(drop=True, inplace=True)
개회11.reset_index(drop=True, inplace=True)
신복11.reset_index(drop=True, inplace=True)

In [ ]:
#필요 칼럼 추가하기
개회11 = pd.merge(개회11, 원데이터11[["계좌키","주민등록번호","주소","최초연체일"]], on="계좌키", how='left')
신복11 = pd.merge(신복11, 원데이터11[["계좌키", "주소", "현재원금", "은행대출원금"]])

In [ ]:
# 개회 작성
개회 = pd.DataFrame(index=range(len(re_d_keys)))
개회["일련번호"] = range(1,len(re_d_keys)+1)
개회["채권구분"] = "개인회생채권"
개회["채권현황"] = 개회11.인가미인가
개회["차주명"] = 개회11.분납자성명인.apply(lambda x:x[:1]+"\u25CB"+x[2:])
개회["채권번호"] = 개회11.계좌키
개회["주민번호"] = 개회11.주민등록번호.apply(lambda x : x[:8]+"*"*len(x[8:]))
개회["차주주소"] = 개회11.주소
개회["최초연체일"] = 개회11.최초연체일
개회["신청일자"] = 개회11.접수일
개회["개시일자"] = 개회11.개시결정일
개회["인가일자"] = 개회11.변제계획인가일
개회["총변제회차"] = 개회11.총분납회차
개회["납입회차"] = 개회11.납입회차
개회["잔여회차"] = 개회11.총분납회차 - 개회11.납입회차
개회["관할법원"] = 개회11.관할법원
개회["사건번호"] = 개회11.사건번호
개회["신청시 인가전 원금"] = ""
개회["Cut-off 현재 대출잔액"] = 개회11.현재원금
개회["Cut-off 현재 가지급금"] = 개회11.현재법비용
개회["확정금액"] = 개회11.총분납금
개회["총입금액"] = 개회11.총분납입금
개회["인가잔여금"] = "수식"
개회["미상환원금잔액(OPB)"] = "수식"
개회["비고"] = 개회11.분납메모 + " // " + 개회11.분납비고

In [ ]:
# 신용회복 작성 - 앞부분 순서가 조금 다르고, 추가된 열이 있긴하나 기본적으로 약식의 칼럼을 따름
신복 = pd.DataFrame(index=range(len(credit_d_keys)))
신복["일련번호"] = range(1,len(credit_d_keys)+1)
신복["채권구분"] = "신용회복채권"
신복["채권현황"] = 신복11.진행구분
신복["채무자명"] = 신복11.신청인명.apply(lambda x:x[:1]+"\u25CB"+x[2:])
신복["채무자주민번호"] = 신복11.신청인주민번호.apply(lambda x : x[:8]+"*"*len(x[8:]))
신복["차주주소"] = 신복11.주소
신복["채권일련번호"] = 신복11.계좌키
신복[["접수번호","거래일"]] = 신복11[["접수번호", "거래일"]]
신복["미상환원금잔액"] = 신복11["현재원금"]
신복[["심의차수", "신청인진행상태", "계좌진행상태", "실효/완제/합의서포기일자", "접수통지일자", "접수일자", "점포명", 
    "확정자통지일", "개인채무조장합의서체결일(확정일)", "상환개시일"]] = 신복11.loc[:,"심의차수" : "상환개시일"]
신복["최초대출원금"] = 신복11.은행대출원금
신복[["조정전원금", "조정전이자", "조정전연체이자", "조정전비용", "조정전합계", "조정후원금", "조정후이자", "조정후연체이자", "조정후비용", "조정후합계", "원금감면여부",
    "조정대상포함여부", "총상환기한", "유예기간", "원금균등상환기간", "원리균등상환기간", "이자상환기간", "납입회차", "연체기간", "총납입금액"]] = 신복11.loc[:,"조정전원금":"총납입금액"]
신복[["총납입원금", "총납입이자", "총납입기타채무"]] = 신복11[["총납입원금", "총납입이자", "총납입기타채무"]]
신복["상환후잔액(OPB)"] = 신복11.반영OPB
신복["감면율"] = 신복11.감면율
신복[["급여가압류원금", "채무구분", "주채무자주민번호", "주채무자성명"]] = 신복11[["급여가압류원금", "채무구분", "주채무자주민번호", "주채무자성명"]]
신복[["동의요청일", "동의확정일", "재조정여부", "재조정횟수", "수정조정여부", "수정조정횟수"]] = 신복11[["동의요청일", "동의확정일", "재조정여부", "재조정횟수", "수정조정여부", "수정조정횟수"]]
신복[["재조정접수일자", "재조정처리일자", "재조정반송일자", "실효취소요청일자", "실효원장회복일자", "상각구분", "신청구분", "일시납감면율",
    "차상위계층여부", "미납발생일", "부동산담보대출여부", "거치기간", "감면방식", "상환방식"]] = 신복11.loc[:, "재조정접수일자":"상환방식"]
신복[["담보권실행유예기간", "특별면책", "재난상환유예", "특례지원", "이자율재조정"]] = 신복11[["담보권실행유예기간", "특별면책", "재난상환유예", "특례지원", "이자율재조정"]]

In [ ]:
# 무담보 작성
무담보 = pd.DataFrame(index=range(len(무담보11)))

무담보["일련번호"] = range(1,len(무담보11)+1)
무담보["채권구분"] = "무담보"
무담보["개인/법인"] = 무담보11.개인법인
무담보["차주명"] = 무담보11.성명.apply(lambda x:x[:1]+"\u25CB"+x[2:])
무담보["채권번호"] = 무담보11.계좌키
무담보["주민번호/사업자번호"] = 무담보11.주민등록번호.apply(lambda x : x[:8]+"*"*len(x[8:]))
무담보["차주주소"] = 무담보11.주소
무담보["매입일"] = 무담보11.매입일
무담보["최초대출일"] = 무담보11.최초대출일
무담보["최초대출액"] = 무담보11.은행대출원금
무담보["대출잔액"] = 무담보11.반영OPB
무담보["시효완성일"] = 무담보11.시효완성일
무담보["비고"] = 무담보11.시효완성월

In [ ]:
# 회수내역 붙이기
####################################################
한투dir = r"D:\3.자산\POOL별 관리자산\솔림\POOL 11 번자료 (베델-01, 한국투자저축)\240930\DATADISK_한투전송"
####################################################
입금11 = pd.read_excel(functions.키워드로파일명찾기(join(한투dir), "입금조회새창_"), dtype = deposit_dtype)

In [ ]:
# 입금구분 정리 및 복사
입금 = 입금11[~입금11.입금구분.str.contains("부채|증명|해지|환매|매각|기타")].copy()
입금["입금월"] = 입금.입금일.apply(lambda x:x[:7])

In [ ]:
# 피벗테이블
월별입금 = 입금[["계좌키", "입금월", "입금합계"]].pivot_table(index="계좌키", columns='입금월', values='입금합계', aggfunc='sum')

In [ ]:
# merge
개회_최종 = pd.merge(개회, 월별입금, left_on='채권번호', right_on ='계좌키', how='left')
신복_최종 = pd.merge(신복, 월별입금, left_on='채권일련번호', right_on ='계좌키', how='left')
무담보_최종 = pd.merge(무담보, 월별입금, left_on='채권번호', right_on ='계좌키', how='left')

# Na값 0으로
개회_최종[월별입금.columns] = 개회_최종[월별입금.columns].fillna(0)
신복_최종[월별입금.columns] = 신복_최종[월별입금.columns].fillna(0)
무담보_최종[월별입금.columns] = 무담보_최종[월별입금.columns].fillna(0)

In [ ]:
# 저장
with pd.ExcelWriter(join(한투dir, "한투_복붙용.xlsx"), engine='openpyxl', mode = 'w') as writer :
    개회_최종.to_excel(writer, sheet_name='개인회생', index=False)
    신복_최종.to_excel(writer, sheet_name='신용회복', index=False)
    무담보_최종.to_excel(writer, sheet_name='무담보', index=False)

# 기타작업

### 시효(계좌)

In [ ]:
wd = r"C:\Users\SL\Desktop\자산정리test파일"
filename_account = "계좌조회새창_20230502_0847.pkl"
PATH_NEW_DEBTKEY = r"C:\Users\SL\Desktop\자산정리test파일\새채무자키.pkl"

account = pd.read_pickle(join(wd,filename_account))
new_debtKey = pd.read_pickle(PATH_NEW_DEBTKEY)
today = datetime.today().strftime("%y%m%d")

In [ ]:
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

In [ ]:
# 필요열만 새로운 df로 & 정렬
account_limit_info = account[["새채무자키", '채무자키','계좌키','채무상태','담당자', "시효완성일", "시효완성월", "최초연체일", "최초시효완성일메모", "시효연장사유", "시효중단여부"]].copy()
account_limit_info.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

In [ ]:
# 동일한 새채무자키 시효중단여부 : 시효가 ##########################################작성중 중단
def set_extension_info(group):
    if 'Y' in group['시효중단여부'].values:
        group.loc[group['시효중단여부'] == '', '시효중단여부'] = 'Y'
        extension_reason = group.loc[group['시효중단여부'] == 'Y', '시효연장사유'].values[0]
        group['시효연장사유'] = extension_reason
    return group

account_limit_info = account_limit_info.groupby('새채무자키').apply(set_extension_info)


### 시효(법조치)

In [ ]:
wd = r"C:\Users\SL\Desktop\자산정리test파일"
filename_event = "법조치조회새창_20230522_2054.xlsx"
dtype_event = {"채무자키":str, "계좌키":str, "법조치키":str}
today = datetime.today().strftime("%y%m%d")

In [ ]:
# 파일 읽기/필요칼럼추리기/원본파일pkl과 필요컬럼xlsx 파일 저장하기
df_event_ori = pd.read_excel(join(wd, filename_event), dtype=dtype_event)

In [ ]:
df_event = pd.DataFrame(None, columns=["채무자키", "계좌키", "법조치키", "법조치구분", "법조치세부", "관할법원", "사건번호", "사건구분", "접수일", "확정일", "종료일",
    "법조치상태", "종국결과", "청구금액법원", "청구금액", "관련법조치관할법원", "관련법조치사건번호"])
    ###################################################### 여기까지 하고 계좌시트 작업으로 넘어감
df_event.채무자키 = df_event_ori.채무자키
df_event.계좌키 = df_event_ori.계좌키
df_event.신청일자 = df_event_ori.접수일
df_event.법무구분 = df_event_ori.법조치구분
df_event.관할법원 = df_event_ori.관할법원
df_event.사건번호 = df_event_ori.사건번호.str.replace(" ", "")
df_event.사건구분 = df_event_ori.사건구분
df_event.진행상태 = df_event_ori.종국결과
df_event.청구금액법원 = df_event_ori.청구금액법원
df_event.청구금액 = df_event_ori.청구금액
print(len(df_event.index))

####################
# 중복제거전 저장
df_event.to_pickle(join(wd, "전처리완료", "전처리_법조치_중복미제거.pkl"))
#df_event.to_excel(join(wd, "전처리완료", "전처리_법조치_중복미제거.xlsx"), index=False)
####################

# 채무자키로 전 계좌 불러오기
work_all_account = all_df[all_df.채무자키.isin(work_df.채무자키)].copy() # 객체 새로 만들지 않으면 계속 copywarning 나옴
# 전 계좌 중 작업 계좌에 있거나, 종결된 계좌면 Y, 아니면 N(솔림 잔존계좌)
cond = (work_all_account.계좌키.isin(work_df.계좌키)) | (work_all_account.종결일 != "")
work_all_account["계좌작업대상여부"] = np.where(cond, "Y", "N")

# 채무자키별로 전계좌 여부 확인 후 파일 저장하기
result = pd.DataFrame(None, columns=["채무자키", "전계좌여부", "채무상태", "담당자", "채무자명"])
temp = work_all_account.drop_duplicates('채무자키').copy()
result["채무자키"] = temp.채무자키.values
result["채무상태"] = temp.채무상태.values # 순서 조작만 하지 않으면 굳이 일치조건 하지 않아도 됨.
result["담당자"] = temp.담당자.values
result["채무자명"] = temp.채무자명.values


# 작업대상여부의 유니크값이 1이면 모두 Y (모두 N인 경우는 논리적으로 불가능), 2면 작업계좌와 잔존계좌가 혼재한 것이므로 일부계좌
g = work_all_account.groupby('채무자키') # 그룹화는 원본 df의 순서와 다를 수 있음!!!!!!!!
for i, v in result.iterrows() : 
     if g.get_group(v.채무자키).계좌작업대상여부.nunique() == 1 :
          result.loc[i, "전계좌여부"]="전계좌"
     else : result.loc[i, "전계좌여부"]="일부계좌"

# 저장하기
result.to_excel(path_save, index=False)